In [ ]:
# https://k3no.medium.com/how-to-query-in-graphql-6ebb3f7085dc
# https://github.com/sushiswap/sushiswap-subgraph

In [1]:
from gql import gql, Client
from gql.transport.requests import RequestsHTTPTransport
import pandas as pd
import dataframe_image as dfi
import os
from datetime import datetime
import numpy as np
pd.set_option('display.max_rows', 99999)
pd.set_option('display.max_columns', 99999)
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [2]:
def client(url):
    sample_transport = RequestsHTTPTransport(url = url,
                                             verify = True,
                                             retries = 3)
    
    client = Client(transport=sample_transport)
    return client

In [3]:
# interesting tokens on MATIC

 #https://polygon.fulcrum.trade/borrow/asset
lending_tokens_fulcrum = ['LINK', 'BZRX', 'AAVE', 'WBTC', 'USDC', 'USDT', 'WETH', 'WMATIC']

# https://creamfinance.substack.com/p/cream-communications-14
lending_tokens_aave = ['MATIC', 'DAI', 'USDC', 'USDT', 'WETH', 'WBTC', 'AAVE']
lending_tokens_cream = ['USDC', 'USDT', 'DAI', 'WMATIC', 'WETH', 'WBTC', 'LINK', 'SUSHI', 'CRV', 'QUICK']
remaining_interesting_tokens = ['FRAX', 'ibBTC']

interesting_tokens = list(set(lending_tokens_fulcrum +\
                              lending_tokens_aave +\
                              lending_tokens_cream +\
                              remaining_interesting_tokens))

interesting_tokens

['LINK',
 'ibBTC',
 'USDT',
 'WETH',
 'SUSHI',
 'AAVE',
 'DAI',
 'WBTC',
 'WMATIC',
 'CRV',
 'BZRX',
 'QUICK',
 'MATIC',
 'FRAX',
 'USDC']

## Quickswap

In [4]:
query = gql('''
query {
  uniswapFactories {
    pairCount
  }
}
''')

response = client('https://api.thegraph.com/subgraphs/name/sameepsi/quickswap02').execute(query)
number_of_pairs = int(response['uniswapFactories'][0]['pairCount'])
print(number_of_pairs)

6196


In [5]:
try:
    df = pd.read_excel('pairdata_Quickswap_MATIC.xlsx')
except:
    df = pd.DataFrame([])
    response_dict = {}
    response_list = []


    for i in range(number_of_pairs):
        if i % 100 == 0:
            querystring = '''
                  query {
                  pairs(first: '''+str(100)+' skip: '+str(i)+''' where: {volumeUSD_gt:0}){
                          token0 {symbol}
                          token1 {symbol}
                          id
                      }
                    }
                    '''
            try:
                response = client.execute(gql(querystring))
                df = df.append(pd.DataFrame.from_dict(response['pairs']))
                print(len(df))
            except:
                pass

In [6]:
if df.columns.tolist() == ['id','Pair', 'Token 0', 'Token 1']:
    pass
else:
    df['token0'] = df['token0'].astype(str).str.replace("{'symbol': ",'', regex= True)\
                                               .replace("}",'', regex= True)\
                                               .replace("'",'', regex= True)
    df['token1'] = df['token1'].astype(str).str.replace("{'symbol': ",'', regex= True)\
                                               .replace("}",'', regex= True)\
                                               .replace("'",'', regex= True)
    df['Pair'] = df['token0'] + '-' + df['token1']
    df.columns = ['id','Token 0', 'Token 1', 'Pair']
    df = df[['id','Pair', 'Token 0', 'Token 1']]
    df.head(2)

In [7]:
criteria = df['Token 0'].isin(interesting_tokens) & \
           df['Token 1'].isin(interesting_tokens)

df_filtered = df.loc[criteria,:]
df_filtered.to_excel('pairdata_Quickswap_MATIC.xlsx', index = False)
df_filtered.head()

,id,Pair,Token 0,Token 1
0,0x019ba0325f1988213d448b3472fa1cf8d07618d7,WMATIC-QUICK,WMATIC,QUICK
1,0x02e0e14d65175420fb12acd325a7be456bc22005,QUICK-DAI,QUICK,DAI
2,0x0554059d42e26f35cc958581c71fdfd92405d02f,WMATIC-AAVE,WMATIC,AAVE
3,0x0b3a76d33d93b2651dbf03b102eefa9500e89243,SUSHI-WMATIC,SUSHI,WMATIC
4,0x1bd06b96dd42ada85fdd0795f3b4a79db914add5,WETH-QUICK,WETH,QUICK


In [8]:
df4 = pd.DataFrame([])
lenid = len(df_filtered['id'])

for count, id in enumerate(df_filtered['id']):
    print(f'{count+1:2} / {lenid} pair: {id}')

    querystring = '''
                  query {
      pairs(where: {id:"''' + id + '''"}){
        volumeUSD
      }
    }'''
    response = client('https://api.thegraph.com/subgraphs/name/moonster-bsc/quickswap-daypairdataslim')\
                      .execute(gql(querystring))

    df1 = pd.DataFrame.from_dict(response['pairs'])
    df1['id'] = id
    df1['timestamp'] = datetime.now()
    df1['volumeUSD'] = df1['volumeUSD'].astype(float)
    df4 = df4.append(df1)
    
df4 = df4[['id', 'timestamp', 'volumeUSD']]

 1 / 47 pair: 0x019ba0325f1988213d448b3472fa1cf8d07618d7
 2 / 47 pair: 0x02e0e14d65175420fb12acd325a7be456bc22005
 3 / 47 pair: 0x0554059d42e26f35cc958581c71fdfd92405d02f
 4 / 47 pair: 0x0b3a76d33d93b2651dbf03b102eefa9500e89243
 5 / 47 pair: 0x1bd06b96dd42ada85fdd0795f3b4a79db914add5
 6 / 47 pair: 0x1f1e4c845183ef6d50e9609f16f6f9cae43bc9cb
 7 / 47 pair: 0x2aa7a18ceabf2ef893d2f7c0145cc45e6f10b223
 8 / 47 pair: 0x2cf7252e74036d1da831d11089d326296e64a728
 9 / 47 pair: 0x3c986748414a812e455dcd5418246b8fded5c369
10 / 47 pair: 0x4a35582a710e1f4b2030a3f826da20bfb6703c09
11 / 47 pair: 0x4f2cc25028d4d80355c802482463b5b2a93cdb84
12 / 47 pair: 0x527a26525f5483612a1f96cfceacc7032b1c8610
13 / 47 pair: 0x59153f27eefe07e5ece4f9304ebba1da6f53ca88
14 / 47 pair: 0x5ca6ca6c3709e1e6cfe74a50cf6b2b6ba2dadd67
15 / 47 pair: 0x600b588d7e847665b2f36cdc70838d04ef9790c2
16 / 47 pair: 0x604229c960e5cacf2aaeac8be68ac07ba9df81c3
17 / 47 pair: 0x609360d5f6dcc26597165ac76e432a16fc27b6fd
18 / 47 pair: 0x634f9d9f8680349

In [9]:
df_old = pd.read_csv("Quickswap export.csv",
                     usecols = ['id', 'timestamp', 'volumeUSD'])
df_old = df_old[['id', 'timestamp', 'volumeUSD']]
df_old['timestamp'] = pd.to_datetime(df_old['timestamp'])
df_old.columns = ['id_old', 'timestamp_old', 'volumeUSD_old']
df_old.head(1)

,id_old,timestamp_old,volumeUSD_old
0,0x019ba0325f1988213d448b3472fa1cf8d07618d7,2021-07-06 18:03:16.515651,8.670924e+08


In [10]:
df4.to_csv("Quickswap export.csv", index = False)

In [11]:
df_combined = pd.concat([df4.set_index('id'), df_old.set_index('id_old')], axis = 1)
df_combined['timedelta'] = (df_combined['timestamp'] - df_combined['timestamp_old']) / np.timedelta64(1, 'Y')
df_combined['volumeUSD delta'] = df_combined['volumeUSD'] - df_combined['volumeUSD_old']
df_combined['fees per year'] = df_combined['volumeUSD delta'] * 0.003 / df_combined['timedelta']
df_combined.head(3)

,timestamp,volumeUSD,timestamp_old,volumeUSD_old,timedelta,volumeUSD delta,fees per year
0x019ba0325f1988213d448b3472fa1cf8d07618d7,2021-07-07 08:01:58.644045,8.685917e+08,2021-07-06 18:03:16.515651,8.670924e+08,0.001595,1.499329e+06,2.820683e+06
0x02e0e14d65175420fb12acd325a7be456bc22005,2021-07-07 08:01:58.849527,2.317646e+05,2021-07-06 18:03:16.720562,2.313270e+05,0.001595,4.376013e+02,8.232579e+02
0x0554059d42e26f35cc958581c71fdfd92405d02f,2021-07-07 08:01:59.051294,4.373444e+05,2021-07-06 18:03:16.917589,4.366511e+05,0.001595,6.933237e+02,1.304347e+03


In [12]:
df4 = pd.DataFrame([])
lenid = len(df_filtered['id'])

for count, id in enumerate(df_filtered['id']):
    print(f'{count+1:2} / {lenid} pair: {id}')
        
    querystring = '''
              query {
  pairDayDatas(where: {pairAddress:"''' + id + '''"} orderBy: date, orderDirection: desc first: 2) {
    date
    dailyVolumeUSD
    reserveUSD
  }
}'''
    response = client('https://api.thegraph.com/subgraphs/name/moonster-bsc/quickswap-daypairdataslim')\
                      .execute(gql(querystring))
    
    df1 = pd.DataFrame.from_dict(response['pairDayDatas'])
    df1['id'] = id
    df4 = df4.append(df1)

 1 / 47 pair: 0x019ba0325f1988213d448b3472fa1cf8d07618d7
 2 / 47 pair: 0x02e0e14d65175420fb12acd325a7be456bc22005
 3 / 47 pair: 0x0554059d42e26f35cc958581c71fdfd92405d02f
 4 / 47 pair: 0x0b3a76d33d93b2651dbf03b102eefa9500e89243
 5 / 47 pair: 0x1bd06b96dd42ada85fdd0795f3b4a79db914add5
 6 / 47 pair: 0x1f1e4c845183ef6d50e9609f16f6f9cae43bc9cb
 7 / 47 pair: 0x2aa7a18ceabf2ef893d2f7c0145cc45e6f10b223
 8 / 47 pair: 0x2cf7252e74036d1da831d11089d326296e64a728
 9 / 47 pair: 0x3c986748414a812e455dcd5418246b8fded5c369
10 / 47 pair: 0x4a35582a710e1f4b2030a3f826da20bfb6703c09
11 / 47 pair: 0x4f2cc25028d4d80355c802482463b5b2a93cdb84
12 / 47 pair: 0x527a26525f5483612a1f96cfceacc7032b1c8610
13 / 47 pair: 0x59153f27eefe07e5ece4f9304ebba1da6f53ca88
14 / 47 pair: 0x5ca6ca6c3709e1e6cfe74a50cf6b2b6ba2dadd67
15 / 47 pair: 0x600b588d7e847665b2f36cdc70838d04ef9790c2
16 / 47 pair: 0x604229c960e5cacf2aaeac8be68ac07ba9df81c3
17 / 47 pair: 0x609360d5f6dcc26597165ac76e432a16fc27b6fd
18 / 47 pair: 0x634f9d9f8680349

In [13]:
df5 = df4.merge(df_filtered,
                left_on = 'id',
                right_on = 'id',
                how = 'left',
                validate = 'many_to_one')
df5[['dailyVolumeUSD','reserveUSD']] = df5[['dailyVolumeUSD','reserveUSD']].astype(float)
df5['date'] = pd.to_datetime(df5['date'], unit='s').dt.date
df5['fee'] = round(df5['dailyVolumeUSD'] * 0.003, 6)
df5['APR yday'] = round(100 * df5['fee'] * 365 / df5['reserveUSD'], 3)
df5['Pair'] = '[Quickswap MATIC] ' + df5['Pair']
df5['APR yday Liq + 200'] = round(100 * df5['fee'] * 365 / (df5['reserveUSD'] + 200), 3)
df5['APR yday Liq + 500'] = round(100 * df5['fee'] * 365 / (df5['reserveUSD'] + 500), 3)
df5['APR yday Liq + 1000'] = round(100 * df5['fee'] * 365 / (df5['reserveUSD'] + 1000), 3)
df5['APR yday Liq + 2000'] = round(100 * df5['fee'] * 365 / (df5['reserveUSD'] + 2000), 3)
df5['APR yday Liq - 200'] = round(100 * df5['fee'] * 365 / (df5['reserveUSD'] - 200), 3)
df5['APR yday Liq - 500'] = round(100 * df5['fee'] * 365 / (df5['reserveUSD'] - 500), 3)
df5['APR yday Liq - 1000'] = round(100 * df5['fee'] * 365 / (df5['reserveUSD'] - 1000), 3)
df5['APR yday Liq - 2000'] = round(100 * df5['fee'] * 365 / (df5['reserveUSD'] - 2000), 3)

df5.head(2)

,dailyVolumeUSD,date,reserveUSD,id,Pair,Token 0,Token 1,fee,APR yday,APR yday Liq + 200,APR yday Liq + 500,APR yday Liq + 1000,APR yday Liq + 2000,APR yday Liq - 200,APR yday Liq - 500,APR yday Liq - 1000,APR yday Liq - 2000
0,6.370877e+05,2021-07-07,2.599264e+07,0x019ba0325f1988213d448b3472fa1cf8d07618d7,[Quickswap MATIC] WMATIC-QUICK,WMATIC,QUICK,1911.263135,2.684,2.684,2.684,2.684,2.684,2.684,2.684,2.684,2.684
1,2.862175e+06,2021-07-06,2.430046e+07,0x019ba0325f1988213d448b3472fa1cf8d07618d7,[Quickswap MATIC] WMATIC-QUICK,WMATIC,QUICK,8586.526485,12.897,12.897,12.897,12.897,12.896,12.897,12.897,12.898,12.898


In [14]:
latest_results = df5.sort_values(by = 'date',
                                 ascending = False)\
                    .drop_duplicates(subset = ['Pair'], keep = 'last')\
                    .reset_index(drop = True)\
                    .drop(columns = ['Token 0', 'Token 1'])\
                    .set_index('id')

latest_results.head(2)

,dailyVolumeUSD,date,reserveUSD,Pair,fee,APR yday,APR yday Liq + 200,APR yday Liq + 500,APR yday Liq + 1000,APR yday Liq + 2000,APR yday Liq - 200,APR yday Liq - 500,APR yday Liq - 1000,APR yday Liq - 2000
id,,,,,,,,,,,,,,
0xfba51e1ea2d46c4d5e98f27e1f5333969d2222d7,0.321629,2021-07-06,11.583302,[Quickswap MATIC] LINK-DAI,0.000965,3.041,0.166,0.069,0.035,0.018,-0.187,-0.072,-0.036,-0.018
0xab4f7a3199b2cd7a8bfb425941ead08facb6d617,5.654882,2021-07-06,18.952447,[Quickswap MATIC] CRV-WETH,0.016965,32.672,2.828,1.193,0.608,0.307,-3.420,-1.287,-0.631,-0.313


In [15]:
df_combined2 = pd.concat([df_combined,
                          df5.sort_values(by = 'date', ascending = False)\
                             .drop_duplicates(subset = ['Pair'],
                                              keep = 'first')\
                             .set_index('id')['reserveUSD']],
                         axis = 1)
df_combined2['APR timedelta'] = 100 * df_combined2['fees per year'] / df_combined2['reserveUSD']
df_combined2['APR timedelta Liq + 200'] = 100 * df_combined2['fees per year'] / (df_combined2['reserveUSD'] + 200)
df_combined2['APR timedelta Liq + 500'] = 100 * df_combined2['fees per year'] / (df_combined2['reserveUSD'] + 500)
df_combined2['APR timedelta Liq + 1000'] = 100 * df_combined2['fees per year'] / (df_combined2['reserveUSD'] + 1000)
df_combined2['APR timedelta Liq + 2000'] = 100 * df_combined2['fees per year'] / (df_combined2['reserveUSD'] + 2000)
df_combined2['APR timedelta Liq - 200'] = 100 * df_combined2['fees per year'] / (df_combined2['reserveUSD'] - 200)
df_combined2['APR timedelta Liq - 500'] = 100 * df_combined2['fees per year'] / (df_combined2['reserveUSD'] - 500)
df_combined2['APR timedelta Liq - 1000'] = 100 * df_combined2['fees per year'] / (df_combined2['reserveUSD'] - 1000)
df_combined2['APR timedelta Liq - 2000'] = 100 * df_combined2['fees per year'] / (df_combined2['reserveUSD'] - 2000)

df_combined2 = df_combined2[['APR timedelta',
                             'APR timedelta Liq + 200',
                             'APR timedelta Liq + 500',
                             'APR timedelta Liq + 1000',                             
                             'APR timedelta Liq + 2000',
                             'APR timedelta Liq - 200',
                             'APR timedelta Liq - 500',
                             'APR timedelta Liq - 1000',                             
                             'APR timedelta Liq - 2000',                             
                             'reserveUSD']]
df_combined2.head(2)

,APR timedelta,APR timedelta Liq + 200,APR timedelta Liq + 500,APR timedelta Liq + 1000,APR timedelta Liq + 2000,APR timedelta Liq - 200,APR timedelta Liq - 500,APR timedelta Liq - 1000,APR timedelta Liq - 2000,reserveUSD
0x019ba0325f1988213d448b3472fa1cf8d07618d7,10.851850,10.851766,10.851641,10.851433,10.851015,10.851933,10.852059,10.852267,10.852685,2.599264e+07
0x02e0e14d65175420fb12acd325a7be456bc22005,10.486846,10.226316,9.858920,9.301944,8.357621,10.760998,11.200200,12.017687,14.071856,7.850386e+03


In [16]:

df_final_quickswap = pd.concat([df_combined2, latest_results],
                                 axis = 1)[['Pair',
                                            'date',
                                            'APR timedelta',
                                            'APR timedelta Liq + 200',
                                            'APR timedelta Liq + 500',
                                            'APR timedelta Liq + 1000',                                            
                                            'APR timedelta Liq + 2000',
                                            'APR timedelta Liq - 200',
                                            'APR timedelta Liq - 500',
                                            'APR timedelta Liq - 1000',                                            
                                            'APR timedelta Liq - 2000',                                             
                                            'APR yday',
                                            'APR yday Liq + 200',
                                            'APR yday Liq + 500',
                                            'APR yday Liq + 1000',                                            
                                            'APR yday Liq + 2000',
                                            'APR yday Liq - 200',
                                            'APR yday Liq - 500',
                                            'APR yday Liq - 1000',                                            
                                            'APR yday Liq - 2000',                                            
                                            'reserveUSD',
                                            'dailyVolumeUSD']].set_index('Pair')\
                       .sort_values(by = 'APR timedelta Liq + 2000', ascending = False)

df_final_quickswap.columns = ['date',
                              'APR timedelta',
                              'APR timedelta Liq + 200',
                              'APR timedelta Liq + 500',
                              'APR timedelta Liq + 1000',                                            
                              'APR timedelta Liq + 2000',
                              'APR timedelta Liq - 200',
                              'APR timedelta Liq - 500',
                              'APR timedelta Liq - 1000',                                            
                              'APR timedelta Liq - 2000',                            
                              'APR yday',
                              'APR yday Liq + 200',
                              'APR yday Liq + 500',
                              'APR yday Liq + 1000',                                            
                              'APR yday Liq + 2000',
                              'APR yday Liq - 200',
                              'APR yday Liq - 500',
                              'APR yday Liq - 1000',                                            
                              'APR yday Liq - 2000',                              
                              'Liquidity timedelta',
                              'Liquidity yday',
                              'Volume yday']

df_final_quickswap.head(1)

# df_final_quickswap_stylized = df_final_quickswap.dropna()\
#                                                 .style.format({'APR timedelta': "{:0<4,.2f}",
#                                                                'APR timedelta Liq + 100': "{:0<4,.2f}",
#                                                            'APR yday': "{:0<4,.2f}",
#                                                            'APR yday Liq + 100': "{:0<4,.2f}",
#                                                            'Liquidity timedelta': "{:0<4,.0f}",
#                                                            'Liquidity yday': "{:0<4,.0f}",
#                                                            'Volume yday': "{:0<4,.0f}"})

# df_final_quickswap_stylized

,date,APR timedelta,APR timedelta Liq + 200,APR timedelta Liq + 500,APR timedelta Liq + 1000,APR timedelta Liq + 2000,APR timedelta Liq - 200,APR timedelta Liq - 500,APR timedelta Liq - 1000,APR timedelta Liq - 2000,APR yday,APR yday Liq + 200,APR yday Liq + 500,APR yday Liq + 1000,APR yday Liq + 2000,APR yday Liq - 200,APR yday Liq - 500,APR yday Liq - 1000,APR yday Liq - 2000,Liquidity timedelta,Liquidity yday,Volume yday
Pair,,,,,,,,,,,,,,,,,,,,,,
[Quickswap MATIC] WMATIC-USDT,2021-07-06,42.729459,42.721247,42.708935,42.688432,42.647484,42.737673,42.750002,42.770564,42.811749,56.635,56.625,56.611,56.587,56.538,56.645,56.659,56.684,56.732,1.040499e+06,1.165712e+06,602924.484385


In [17]:
# dfi.export(df_final_quickswap_stylized, 'Quickswap MATIC timedelta.png')
# os.system('xdg-open "Quickswap MATIC timedelta.png"')

## Sushiswap MATIC

In [18]:
# sushiswap_matic = 'https://api.thegraph.com/subgraphs/name/sushiswap/matic-exchange'

# sample_transport=RequestsHTTPTransport(url = sushiswap_matic,
#                                        verify = True,
#                                        retries = 3)

# client = Client(transport=sample_transport)

In [19]:
query = gql('''
query {
  factories {
    pairCount
  }
}
''')

response = client('https://api.thegraph.com/subgraphs/name/sushiswap/matic-exchange').execute(query)
number_of_pairs = int(response['factories'][0]['pairCount'])
print(number_of_pairs)

1329


In [20]:
try:
    df = pd.read_excel('pairdata_SushiSwap_MATIC.xlsx')
except:
    df = pd.DataFrame([])
    response_dict = {}
    response_list = []


    for i in range(number_of_pairs):
        if i % 100 == 0:
            querystring = '''
                  query {
                  pairs(first: '''+str(100)+' skip: '+str(i)+''' where: {volumeUSD_gt:0}){
                          token0 {symbol}
                          token1 {symbol}
                          id
                      }
                    }
                    '''
            try:
                response = client.execute(gql(querystring))
                df = df.append(pd.DataFrame.from_dict(response['pairs']))
                print(len(df))
            except:
                pass

In [21]:
if df.columns.tolist() == ['id','Pair', 'Token 0', 'Token 1']:
    pass
else:
    df['token0'] = df['token0'].astype(str).str.replace("{'symbol': ",'', regex= True)\
                                               .replace("}",'', regex= True)\
                                               .replace("'",'', regex= True)
    df['token1'] = df['token1'].astype(str).str.replace("{'symbol': ",'', regex= True)\
                                               .replace("}",'', regex= True)\
                                               .replace("'",'', regex= True)
    df['Pair'] = df['token0'] + '-' + df['token1']
    df.columns = ['id','Token 0', 'Token 1', 'Pair']
    df = df[['id','Pair', 'Token 0', 'Token 1']]
    df.head(2)

In [22]:
criteria = df['Token 0'].isin(interesting_tokens) & \
           df['Token 1'].isin(interesting_tokens)

df_filtered = df.loc[criteria,:]
df_filtered.to_excel("pairdata_SushiSwap_MATIC.xlsx", index = False)
df_filtered.head()

,id,Pair,Token 0,Token 1
0,0x140ea3fae80a2732ebc4de0511ff84ef1a575217,WMATIC-CRV,WMATIC,CRV
1,0x177e42f6b3da3ffdda3dcbb6010f00a380c3cb05,SUSHI-DAI,SUSHI,DAI
2,0x264a05020b491ba1969a325b0a474a864bb0d93b,FRAX-USDT,FRAX,USDT
3,0x2813d43463c374a680f235c428fb1d7f08de0b69,WETH-AAVE,WETH,AAVE
4,0x29bb3ddf61c831adda06439740b340ae48606da6,WBTC-QUICK,WBTC,QUICK


In [23]:
df4 = pd.DataFrame([])
lenid = len(df_filtered['id'])

for count, id in enumerate(df_filtered['id']):
    print(f'{count+1:2} / {lenid} pair: {id}')

    querystring = '''
                  query {
      pairs(where: {id:"''' + id + '''"}){
        volumeUSD
      }
    }'''
    response = client('https://api.thegraph.com/subgraphs/name/sushiswap/matic-exchange').execute(gql(querystring))

    df1 = pd.DataFrame.from_dict(response['pairs'])
    df1['id'] = id
    df1['timestamp'] = datetime.now()
    df1['volumeUSD'] = df1['volumeUSD'].astype(float)
    df4 = df4.append(df1)
    
df4 = df4[['id', 'timestamp', 'volumeUSD']]
df4.head(1)

 1 / 44 pair: 0x140ea3fae80a2732ebc4de0511ff84ef1a575217
 2 / 44 pair: 0x177e42f6b3da3ffdda3dcbb6010f00a380c3cb05
 3 / 44 pair: 0x264a05020b491ba1969a325b0a474a864bb0d93b
 4 / 44 pair: 0x2813d43463c374a680f235c428fb1d7f08de0b69
 5 / 44 pair: 0x29bb3ddf61c831adda06439740b340ae48606da6
 6 / 44 pair: 0x34965ba0ac2451a34a0471f04cca3f990b8dea27
 7 / 44 pair: 0x383be588327216586e131e63592a2dc976a16655
 8 / 44 pair: 0x396e655c309676caf0acf4607a868e0cded876db
 9 / 44 pair: 0x3b31bb4b6ba4f67f4ef54e78bcb0aaa4f53dc7ff
10 / 44 pair: 0x3bffd4ca16197ad48dc610c41a022664905b7ffc
11 / 44 pair: 0x3e9a0586088caeb2adff32b0c06cfa44151c7a5b
12 / 44 pair: 0x4b1f1e2435a9c96f7330faea190ef6a7c8d70001
13 / 44 pair: 0x55ff76bffc3cdd9d5fdbbc2ece4528ecce45047e
14 / 44 pair: 0x597a9bc3b24c2a578ccb3aa2c2c62c39427c6a49
15 / 44 pair: 0x5c26e61c6153805b073beb185319de61adbdcb98
16 / 44 pair: 0x5f3eeceead64ee34b78e80cd574496e82e537541
17 / 44 pair: 0x68cce7049013ca8df91cd512cefee8fc0bb8d926
18 / 44 pair: 0x6ff62bfb8c12109

,id,timestamp,volumeUSD
0,0x140ea3fae80a2732ebc4de0511ff84ef1a575217,2021-07-07 08:04:09.441710,1072.825716


In [24]:
df_old = pd.read_csv("SushiSwap export.csv",
                     usecols = ['id', 'timestamp', 'volumeUSD'])
df_old = df_old[['id', 'timestamp', 'volumeUSD']]
df_old['timestamp'] = pd.to_datetime(df_old['timestamp'])
df_old.columns = ['id_old', 'timestamp_old', 'volumeUSD_old']
df_old.head(1)

,id_old,timestamp_old,volumeUSD_old
0,0x140ea3fae80a2732ebc4de0511ff84ef1a575217,2021-07-06 18:05:33.112708,1041.63398


In [25]:
df4.to_csv("SushiSwap export.csv", index = False)

In [26]:
df_combined = pd.concat([df4.set_index('id'), df_old.set_index('id_old')], axis = 1)
df_combined['timedelta'] = (df_combined['timestamp'] - df_combined['timestamp_old']) / np.timedelta64(1, 'Y')
df_combined['volumeUSD delta'] = df_combined['volumeUSD'] - df_combined['volumeUSD_old']
df_combined['fees per year'] = df_combined['volumeUSD delta'] * 0.003 / df_combined['timedelta']
df_combined.head(2)

,timestamp,volumeUSD,timestamp_old,volumeUSD_old,timedelta,volumeUSD delta,fees per year
0x140ea3fae80a2732ebc4de0511ff84ef1a575217,2021-07-07 08:04:09.441710,1072.825716,2021-07-06 18:05:33.112708,1041.633980,0.001594,31.191736,58.687674
0x177e42f6b3da3ffdda3dcbb6010f00a380c3cb05,2021-07-07 08:04:09.651578,10023.101923,2021-07-06 18:05:33.330484,9982.200924,0.001594,40.900999,76.955797


In [27]:
df4 = pd.DataFrame([])
lenid = len(df_filtered['id'])

for count, id in enumerate(df_filtered['id']):
    print(f'{count+1:2} / {lenid} pair: {id}')
        
    querystring = '''
              query {
  pairDayDatas(where: {pair:"''' + id + '''"} orderBy: date, orderDirection: desc first: 2) {
    date
    volumeUSD
    reserveUSD
  }
}'''
    response = client('https://api.thegraph.com/subgraphs/name/sushiswap/matic-exchange').execute(gql(querystring))
    response
    
    df1 = pd.DataFrame.from_dict(response['pairDayDatas'])
    df1['id'] = id
    df4 = df4.append(df1)

 1 / 44 pair: 0x140ea3fae80a2732ebc4de0511ff84ef1a575217
 2 / 44 pair: 0x177e42f6b3da3ffdda3dcbb6010f00a380c3cb05
 3 / 44 pair: 0x264a05020b491ba1969a325b0a474a864bb0d93b
 4 / 44 pair: 0x2813d43463c374a680f235c428fb1d7f08de0b69
 5 / 44 pair: 0x29bb3ddf61c831adda06439740b340ae48606da6
 6 / 44 pair: 0x34965ba0ac2451a34a0471f04cca3f990b8dea27
 7 / 44 pair: 0x383be588327216586e131e63592a2dc976a16655
 8 / 44 pair: 0x396e655c309676caf0acf4607a868e0cded876db
 9 / 44 pair: 0x3b31bb4b6ba4f67f4ef54e78bcb0aaa4f53dc7ff
10 / 44 pair: 0x3bffd4ca16197ad48dc610c41a022664905b7ffc
11 / 44 pair: 0x3e9a0586088caeb2adff32b0c06cfa44151c7a5b
12 / 44 pair: 0x4b1f1e2435a9c96f7330faea190ef6a7c8d70001
13 / 44 pair: 0x55ff76bffc3cdd9d5fdbbc2ece4528ecce45047e
14 / 44 pair: 0x597a9bc3b24c2a578ccb3aa2c2c62c39427c6a49
15 / 44 pair: 0x5c26e61c6153805b073beb185319de61adbdcb98
16 / 44 pair: 0x5f3eeceead64ee34b78e80cd574496e82e537541
17 / 44 pair: 0x68cce7049013ca8df91cd512cefee8fc0bb8d926
18 / 44 pair: 0x6ff62bfb8c12109

In [28]:
df5 = df4.merge(df_filtered,
                left_on = 'id',
                right_on = 'id',
                how = 'left',
                validate = 'many_to_one')
df5[['volumeUSD','reserveUSD']] = df5[['volumeUSD','reserveUSD']].astype(float)
df5['date'] = pd.to_datetime(df5['date'], unit='s').dt.date
df5['fee'] = round(df5['volumeUSD'] * 0.003, 6)
df5['APR yday'] = round(100 * df5['fee'] * 365 / df5['reserveUSD'], 3)
df5['Pair'] = '[Sushiswap MATIC] ' + df5['Pair']
df5['APR yday Liq + 200'] = round(100 * df5['fee'] * 365 / (df5['reserveUSD'] + 200), 3)
df5['APR yday Liq + 500'] = round(100 * df5['fee'] * 365 / (df5['reserveUSD'] + 500), 3)
df5['APR yday Liq + 1000'] = round(100 * df5['fee'] * 365 / (df5['reserveUSD'] + 1000), 3)
df5['APR yday Liq + 2000'] = round(100 * df5['fee'] * 365 / (df5['reserveUSD'] + 2000), 3)
df5['APR yday Liq - 200'] = round(100 * df5['fee'] * 365 / (df5['reserveUSD'] - 200), 3)
df5['APR yday Liq - 500'] = round(100 * df5['fee'] * 365 / (df5['reserveUSD'] - 500), 3)
df5['APR yday Liq - 1000'] = round(100 * df5['fee'] * 365 / (df5['reserveUSD'] - 1000), 3)
df5['APR yday Liq - 2000'] = round(100 * df5['fee'] * 365 / (df5['reserveUSD'] - 2000), 3)

df5.head(2)

,date,reserveUSD,volumeUSD,id,Pair,Token 0,Token 1,fee,APR yday,APR yday Liq + 200,APR yday Liq + 500,APR yday Liq + 1000,APR yday Liq + 2000,APR yday Liq - 200,APR yday Liq - 500,APR yday Liq - 1000,APR yday Liq - 2000
0,2021-07-07,631.257763,11.030133,0x140ea3fae80a2732ebc4de0511ff84ef1a575217,[Sushiswap MATIC] WMATIC-CRV,WMATIC,CRV,0.033090,1.913,1.453,1.068,0.740,0.459,2.801,9.202,-3.275,-0.882
1,2021-07-06,612.446136,60.338553,0x140ea3fae80a2732ebc4de0511ff84ef1a575217,[Sushiswap MATIC] WMATIC-CRV,WMATIC,CRV,0.181016,10.788,8.132,5.939,4.098,2.529,16.019,58.758,-17.048,-4.762


In [29]:
latest_results = df5.sort_values(by = 'date',
                                 ascending = False)\
                    .drop_duplicates(subset = ['Pair'], keep = 'last')\
                    .reset_index(drop = True)\
                    .drop(columns = ['Token 0', 'Token 1'])\
                    .set_index('id')

latest_results.head(2)

,date,reserveUSD,volumeUSD,Pair,fee,APR yday,APR yday Liq + 200,APR yday Liq + 500,APR yday Liq + 1000,APR yday Liq + 2000,APR yday Liq - 200,APR yday Liq - 500,APR yday Liq - 1000,APR yday Liq - 2000
id,,,,,,,,,,,,,,
0x597a9bc3b24c2a578ccb3aa2c2c62c39427c6a49,2021-07-06,3.180204e+05,220853.060277,[Sushiswap MATIC] SUSHI-WMATIC,662.559181,76.044,75.996,75.924,75.805,75.568,76.091,76.163,76.283,76.525
0xb5846453b67d0b4b4ce655930cf6e4129f4416d7,2021-07-06,1.036305e+06,661827.484045,[Sushiswap MATIC] SUSHI-WETH,1985.482452,69.931,69.918,69.898,69.864,69.797,69.945,69.965,69.999,70.066


In [30]:
df_combined2 = pd.concat([df_combined,
                          df5.sort_values(by = 'date', ascending = False)\
                             .drop_duplicates(subset = ['Pair'],
                                              keep = 'first')\
                             .set_index('id')['reserveUSD']],
                         axis = 1)
df_combined2['APR timedelta'] = 100 * df_combined2['fees per year'] / df_combined2['reserveUSD']
df_combined2['APR timedelta Liq + 200'] = 100 * df_combined2['fees per year'] / (df_combined2['reserveUSD'] + 200)
df_combined2['APR timedelta Liq + 500'] = 100 * df_combined2['fees per year'] / (df_combined2['reserveUSD'] + 500)
df_combined2['APR timedelta Liq + 1000'] = 100 * df_combined2['fees per year'] / (df_combined2['reserveUSD'] + 1000)
df_combined2['APR timedelta Liq + 2000'] = 100 * df_combined2['fees per year'] / (df_combined2['reserveUSD'] + 2000)
df_combined2['APR timedelta Liq - 200'] = 100 * df_combined2['fees per year'] / (df_combined2['reserveUSD'] - 200)
df_combined2['APR timedelta Liq - 500'] = 100 * df_combined2['fees per year'] / (df_combined2['reserveUSD'] - 500)
df_combined2['APR timedelta Liq - 1000'] = 100 * df_combined2['fees per year'] / (df_combined2['reserveUSD'] - 1000)
df_combined2['APR timedelta Liq - 2000'] = 100 * df_combined2['fees per year'] / (df_combined2['reserveUSD'] - 2000)
df_combined2 = df_combined2[['APR timedelta',
                             'APR timedelta Liq + 200',
                             'APR timedelta Liq + 500',
                             'APR timedelta Liq + 1000',                             
                             'APR timedelta Liq + 2000',
                             'APR timedelta Liq - 200',
                             'APR timedelta Liq - 500',
                             'APR timedelta Liq - 1000',                             
                             'APR timedelta Liq - 2000',                             
                             'reserveUSD']]
df_combined2.head(2)

,APR timedelta,APR timedelta Liq + 200,APR timedelta Liq + 500,APR timedelta Liq + 1000,APR timedelta Liq + 2000,APR timedelta Liq - 200,APR timedelta Liq - 500,APR timedelta Liq - 1000,APR timedelta Liq - 2000,reserveUSD
0x140ea3fae80a2732ebc4de0511ff84ef1a575217,9.296943,7.060105,5.187825,3.597695,2.230404,13.608491,44.711774,-15.915637,-4.287708,631.257763
0x177e42f6b3da3ffdda3dcbb6010f00a380c3cb05,14.480588,10.521120,7.460998,5.025058,3.039999,23.218543,244.761719,-16.423933,-5.240225,531.441108


In [31]:
df_final_sushiswap_matic = pd.concat([df_combined2, latest_results],
                                 axis = 1)[['Pair',
                                            'date',
                                            'APR timedelta',
                                            'APR timedelta Liq + 200',
                                            'APR timedelta Liq + 500',
                                            'APR timedelta Liq + 1000',                                            
                                            'APR timedelta Liq + 2000',
                                            'APR timedelta Liq - 200',
                                            'APR timedelta Liq - 500',
                                            'APR timedelta Liq - 1000',                                            
                                            'APR timedelta Liq - 2000',                                             
                                            'APR yday',
                                            'APR yday Liq + 200',
                                            'APR yday Liq + 500',
                                            'APR yday Liq + 1000',                                            
                                            'APR yday Liq + 2000',
                                            'APR yday Liq - 200',
                                            'APR yday Liq - 500',
                                            'APR yday Liq - 1000',                                            
                                            'APR yday Liq - 2000',                                            
                                            'reserveUSD',
                                            'volumeUSD']].set_index('Pair')\
                       .sort_values(by = 'APR timedelta Liq + 2000', ascending = False)

df_final_sushiswap_matic.columns = ['date',
                              'APR timedelta',
                              'APR timedelta Liq + 200',
                                'APR timedelta Liq + 500',
                                'APR timedelta Liq + 1000',                                            
                                'APR timedelta Liq + 2000',
                                'APR timedelta Liq - 200',
                                'APR timedelta Liq - 500',
                                'APR timedelta Liq - 1000',                                            
                                'APR timedelta Liq - 2000',                           
                              'APR yday',
                             'APR yday Liq + 200',
                                'APR yday Liq + 500',
                                'APR yday Liq + 1000',                                            
                                'APR yday Liq + 2000',
                                'APR yday Liq - 200',
                                'APR yday Liq - 500',
                                'APR yday Liq - 1000',                                            
                                'APR yday Liq - 2000',                          
                              'Liquidity timedelta',
                              'Liquidity yday',
                              'Volume yday']

df_final_sushiswap_matic.head(1)

# df_final_sushiswap_matic_stylized = df_final_sushiswap_matic.dropna()\
#                                                 .style.format({'APR timedelta': "{:0<4,.2f}",
#                                                                'APR timedelta Liq + 100': "{:0<4,.2f}",
#                                                            'APR yday': "{:0<4,.2f}",
#                                                            'APR yday Liq + 100': "{:0<4,.2f}",
#                                                            'Liquidity timedelta': "{:0<4,.0f}",
#                                                            'Liquidity yday': "{:0<4,.0f}",
#                                                            'Volume yday': "{:0<4,.0f}"})

# df_final_sushiswap_matic_stylized

,date,APR timedelta,APR timedelta Liq + 200,APR timedelta Liq + 500,APR timedelta Liq + 1000,APR timedelta Liq + 2000,APR timedelta Liq - 200,APR timedelta Liq - 500,APR timedelta Liq - 1000,APR timedelta Liq - 2000,APR yday,APR yday Liq + 200,APR yday Liq + 500,APR yday Liq + 1000,APR yday Liq + 2000,APR yday Liq - 200,APR yday Liq - 500,APR yday Liq - 1000,APR yday Liq - 2000,Liquidity timedelta,Liquidity yday,Volume yday
Pair,,,,,,,,,,,,,,,,,,,,,,
[Sushiswap MATIC] WMATIC-QUICK,2021-07-06,138.977958,131.208454,121.056993,107.229887,87.289506,147.72552,163.126834,197.432861,340.757263,142.29,134.038,123.311,108.8,88.071,151.624,168.172,205.564,370.178,3377.524408,3248.765586,4221.603819


In [32]:
# dfi.export(df_final_stylized_sushi, 'SushiSwap MATIC timedelta.png')
# os.system('xdg-open "SushiSwap MATIC timedelta.png"')

## Honeyswap Polygon

In [33]:
query = gql('''
query {
  honeyswapFactories {
    pairCount
  }
}
''')

response = client('https://api.thegraph.com/subgraphs/name/1hive/honeyswap-polygon').execute(query)
number_of_pairs = int(response['honeyswapFactories'][0]['pairCount'])
print(number_of_pairs)

145


In [34]:
try:
    df = pd.read_excel('pairdata_HoneySwap_MATIC.xlsx')
except:
    df = pd.DataFrame([])
    response_dict = {}
    response_list = []


    for i in range(number_of_pairs):
        if i % 100 == 0:
            querystring = '''
                  query {
                  pairs(first: '''+str(100)+' skip: '+str(i)+''' where: {volumeUSD_gt:0}){
                          token0 {symbol}
                          token1 {symbol}
                          id
                      }
                    }
                    '''
            try:
                response = client('https://api.thegraph.com/subgraphs/name/1hive/honeyswap-polygon').execute(gql(querystring))
                df = df.append(pd.DataFrame.from_dict(response['pairs']))
                print(len(df))
            except:
                pass

In [35]:
if df.columns.tolist() == ['id','Pair', 'Token 0', 'Token 1']:
    pass
else:
    df['token0'] = df['token0'].astype(str).str.replace("{'symbol': ",'', regex= True)\
                                               .replace("}",'', regex= True)\
                                               .replace("'",'', regex= True)
    df['token1'] = df['token1'].astype(str).str.replace("{'symbol': ",'', regex= True)\
                                               .replace("}",'', regex= True)\
                                               .replace("'",'', regex= True)
    df['Pair'] = df['token0'] + '-' + df['token1']
    df.columns = ['id','Token 0', 'Token 1', 'Pair']
    df = df[['id','Pair', 'Token 0', 'Token 1']]
    df.head(2)

In [36]:
criteria = df['Token 0'].isin(interesting_tokens) & \
           df['Token 1'].isin(interesting_tokens)

df_filtered = df.loc[criteria,:]
df_filtered.to_excel("pairdata_HoneySwap_MATIC.xlsx", index = False)
df_filtered.head(2)

,id,Pair,Token 0,Token 1
0,0x0c787944946d22922c9f41c477cc539700d35bb2,WETH-AAVE,WETH,AAVE
1,0x1013ba833071fd8aca8bf2ac83e5515c5fb9e76f,WETH-USDT,WETH,USDT


In [37]:
df_timedelta = pd.DataFrame([])
lenid = len(df_filtered['id'])

for count, id in enumerate(df_filtered['id']):
    print(f'{count+1:2} / {lenid} pair: {id}')

    querystring = '''
                  query {
      pairs(where: {id:"''' + id + '''"}){
        volumeUSD
      }
    }'''
    response = client('https://api.thegraph.com/subgraphs/name/1hive/honeyswap-polygon').execute(gql(querystring))

    df1 = pd.DataFrame.from_dict(response['pairs'])
    df1['id'] = id
    df1['timestamp'] = datetime.now()
    df1['volumeUSD'] = df1['volumeUSD'].astype(float)
    df_timedelta = df_timedelta.append(df1)
    
df_timedelta = df_timedelta[['id', 'timestamp', 'volumeUSD']]
df_timedelta.head(1)

 1 / 27 pair: 0x0c787944946d22922c9f41c477cc539700d35bb2
 2 / 27 pair: 0x1013ba833071fd8aca8bf2ac83e5515c5fb9e76f
 3 / 27 pair: 0x1fc4a2523349bd6df30000b923bb1acb3a27051f
 4 / 27 pair: 0x2177ccb01ec6a3a4d567e42d51f85a65e3ac5ab6
 5 / 27 pair: 0x218e468b15469228f35b0e7f88425cd45fb982bd
 6 / 27 pair: 0x287e9a601a257b24f8d718d08612e9d8ead7f5a8
 7 / 27 pair: 0x35fbfcd7256b3d6bffc69b407e42a135280c0a8b
 8 / 27 pair: 0x3a968d1fe0246de743b6f102e679c11be36688a9
 9 / 27 pair: 0x3f24e142fbf05d16ab1cf79e6df3473f515b16e0
10 / 27 pair: 0x593a7e3c4897834cb6499e051a1ef9532ca82871
11 / 27 pair: 0x5c09be7481c9341a0e4a04843a936429691571a8
12 / 27 pair: 0x6815c440c5c8ad500e1ac6c301f8057b84753268
13 / 27 pair: 0x6d3842ab227a0436a6e8c459e93c74bd8c16fb34
14 / 27 pair: 0x707c48f37a2dacdb729a6880e1c94768d56bbbb0
15 / 27 pair: 0x7fdf6932d8e761c75922af0d4acd04bed36a8c8b
16 / 27 pair: 0x86b7249272fabb82ef36550ef898ea539225e7f0
17 / 27 pair: 0x8df8b187b9a2a7c2d25faa4f33d06c35b25b28bc
18 / 27 pair: 0x96aa3911e89da7b

,id,timestamp,volumeUSD
0,0x0c787944946d22922c9f41c477cc539700d35bb2,2021-07-07 08:04:28.521183,59290.754995


In [38]:
df_old = pd.read_csv("HoneySwap export.csv",
                     usecols = ['id', 'timestamp', 'volumeUSD'])
df_old = df_old[['id', 'timestamp', 'volumeUSD']]
df_old['timestamp'] = pd.to_datetime(df_old['timestamp'])
df_old.columns = ['id_old', 'timestamp_old', 'volumeUSD_old']
df_old.head(1)

,id_old,timestamp_old,volumeUSD_old
0,0x0c787944946d22922c9f41c477cc539700d35bb2,2021-07-06 18:05:52.420763,54266.587027


In [39]:
df_timedelta.to_csv("HoneySwap export.csv", index = False)

In [40]:
df_combined = pd.concat([df_timedelta.set_index('id'), df_old.set_index('id_old')], axis = 1)
df_combined['timedelta'] = (df_combined['timestamp'] - df_combined['timestamp_old']) / np.timedelta64(1, 'Y')
df_combined['volumeUSD delta'] = df_combined['volumeUSD'] - df_combined['volumeUSD_old']
df_combined['fees per year'] = df_combined['volumeUSD delta'] * 0.0025 / df_combined['timedelta']
df_combined.head(2)

,timestamp,volumeUSD,timestamp_old,volumeUSD_old,timedelta,volumeUSD delta,fees per year
0x0c787944946d22922c9f41c477cc539700d35bb2,2021-07-07 08:04:28.521183,59290.754995,2021-07-06 18:05:52.420763,54266.587027,0.001594,5024.167967,7877.569310
0x1013ba833071fd8aca8bf2ac83e5515c5fb9e76f,2021-07-07 08:04:28.716447,128974.409862,2021-07-06 18:05:52.638343,124142.304959,0.001594,4832.104903,7576.430249


In [41]:
df_last_two_days = pd.DataFrame([])
lenid = len(df_filtered['id'])

for count, id in enumerate(df_filtered['id']):
    print(f'{count+1:2} / {lenid} pair: {id}')
        
    querystring = '''
              query {
  pairDayDatas(where: {pairAddress:"''' + id + '''"} orderBy: date, orderDirection: desc first: 2) {
    date
    dailyVolumeUSD
    reserveUSD
  }
}'''
    response = client('https://api.thegraph.com/subgraphs/name/1hive/honeyswap-polygon').execute(gql(querystring))
    response
    
    df1 = pd.DataFrame.from_dict(response['pairDayDatas'])
    df1['id'] = id
    df_last_two_days = df_last_two_days.append(df1)
    
df_last_two_days = df_last_two_days.rename(columns = {'dailyVolumeUSD':'volumeUSD'})

 1 / 27 pair: 0x0c787944946d22922c9f41c477cc539700d35bb2
 2 / 27 pair: 0x1013ba833071fd8aca8bf2ac83e5515c5fb9e76f
 3 / 27 pair: 0x1fc4a2523349bd6df30000b923bb1acb3a27051f
 4 / 27 pair: 0x2177ccb01ec6a3a4d567e42d51f85a65e3ac5ab6
 5 / 27 pair: 0x218e468b15469228f35b0e7f88425cd45fb982bd
 6 / 27 pair: 0x287e9a601a257b24f8d718d08612e9d8ead7f5a8
 7 / 27 pair: 0x35fbfcd7256b3d6bffc69b407e42a135280c0a8b
 8 / 27 pair: 0x3a968d1fe0246de743b6f102e679c11be36688a9
 9 / 27 pair: 0x3f24e142fbf05d16ab1cf79e6df3473f515b16e0
10 / 27 pair: 0x593a7e3c4897834cb6499e051a1ef9532ca82871
11 / 27 pair: 0x5c09be7481c9341a0e4a04843a936429691571a8
12 / 27 pair: 0x6815c440c5c8ad500e1ac6c301f8057b84753268
13 / 27 pair: 0x6d3842ab227a0436a6e8c459e93c74bd8c16fb34
14 / 27 pair: 0x707c48f37a2dacdb729a6880e1c94768d56bbbb0
15 / 27 pair: 0x7fdf6932d8e761c75922af0d4acd04bed36a8c8b
16 / 27 pair: 0x86b7249272fabb82ef36550ef898ea539225e7f0
17 / 27 pair: 0x8df8b187b9a2a7c2d25faa4f33d06c35b25b28bc
18 / 27 pair: 0x96aa3911e89da7b

In [42]:
df_last_two_days2 = df_last_two_days.merge(df_filtered,
                             left_on = 'id',
                             right_on = 'id',
                             how = 'left',
                             validate = 'many_to_one')
df_last_two_days2[['volumeUSD','reserveUSD']] = df_last_two_days2[['volumeUSD','reserveUSD']].astype(float)
df_last_two_days2['date'] = pd.to_datetime(df_last_two_days2['date'], unit='s').dt.date
df_last_two_days2['fee'] = round(df_last_two_days2['volumeUSD'] * 0.0025, 6)
df_last_two_days2['APR yday'] = round(100 * df_last_two_days2['fee'] * 365 / df_last_two_days2['reserveUSD'], 3)
df_last_two_days2['Pair'] = '[Honeyswap MATIC] ' + df_last_two_days2['Pair']
df_last_two_days2['APR yday Liq + 200'] = round(100 * df_last_two_days2['fee'] * 365 / (df_last_two_days2['reserveUSD'] + 200), 3)
df_last_two_days2['APR yday Liq + 500'] = round(100 * df_last_two_days2['fee'] * 365 / (df_last_two_days2['reserveUSD'] + 500), 3) 
df_last_two_days2['APR yday Liq + 1000'] = round(100 * df_last_two_days2['fee'] * 365 / (df_last_two_days2['reserveUSD'] + 1000), 3)                                                  
df_last_two_days2['APR yday Liq + 2000'] = round(100 * df_last_two_days2['fee'] * 365 / (df_last_two_days2['reserveUSD'] + 2000), 3)
df_last_two_days2['APR yday Liq - 200'] = round(100 * df_last_two_days2['fee'] * 365 / (df_last_two_days2['reserveUSD'] - 200), 3)
df_last_two_days2['APR yday Liq - 500'] = round(100 * df_last_two_days2['fee'] * 365 / (df_last_two_days2['reserveUSD'] - 500), 3)
df_last_two_days2['APR yday Liq - 1000'] = round(100 * df_last_two_days2['fee'] * 365 / (df_last_two_days2['reserveUSD'] - 1000), 3)                                                 
df_last_two_days2['APR yday Liq - 2000'] = round(100 * df_last_two_days2['fee'] * 365 / (df_last_two_days2['reserveUSD'] - 2000), 3)                                                 

df_last_two_days2.head(2)

,volumeUSD,date,reserveUSD,id,Pair,Token 0,Token 1,fee,APR yday,APR yday Liq + 200,APR yday Liq + 500,APR yday Liq + 1000,APR yday Liq + 2000,APR yday Liq - 200,APR yday Liq - 500,APR yday Liq - 1000,APR yday Liq - 2000
0,2118.603554,2021-07-07,187156.260133,0x0c787944946d22922c9f41c477cc539700d35bb2,[Honeyswap MATIC] WETH-AAVE,WETH,AAVE,5.296509,1.033,1.032,1.030,1.027,1.022,1.034,1.036,1.038,1.044
1,12161.380873,2021-07-06,181630.722521,0x0c787944946d22922c9f41c477cc539700d35bb2,[Honeyswap MATIC] WETH-AAVE,WETH,AAVE,30.403452,6.110,6.103,6.093,6.076,6.043,6.117,6.127,6.144,6.178


In [43]:
df_yesterday = df_last_two_days2.sort_values(by = 'date',
                                             ascending = False)\
                                .drop_duplicates(subset = ['Pair'], keep = 'last')\
                                .reset_index(drop = True)\
                                .drop(columns = ['Token 0', 'Token 1'])\
                                .set_index('id')

df_yesterday.head(2)

,volumeUSD,date,reserveUSD,Pair,fee,APR yday,APR yday Liq + 200,APR yday Liq + 500,APR yday Liq + 1000,APR yday Liq + 2000,APR yday Liq - 200,APR yday Liq - 500,APR yday Liq - 1000,APR yday Liq - 2000
id,,,,,,,,,,,,,,
0xe49fea624d480a5233b5dfc4969a27319873e6f0,81.214818,2021-07-06,28807.120364,[Honeyswap MATIC] USDC-DAI,0.203037,0.257,0.255,0.253,0.249,0.241,0.259,0.262,0.267,0.276
0xd862db749534d539713b2c392421fe5a8e43e19e,37853.927753,2021-07-06,263193.354972,[Honeyswap MATIC] USDC-WETH,94.634819,13.124,13.114,13.099,13.074,13.025,13.134,13.149,13.174,13.225


In [44]:
df_combined2 = pd.concat([df_combined,
                          df_last_two_days2.sort_values(by = 'date', ascending = False)\
                             .drop_duplicates(subset = ['Pair'],
                                              keep = 'first')\
                             .set_index('id')['reserveUSD']],
                         axis = 1)
df_combined2['APR timedelta'] = 100 * df_combined2['fees per year'] / df_combined2['reserveUSD']
df_combined2['APR timedelta Liq + 200'] = 100 * df_combined2['fees per year'] / (df_combined2['reserveUSD'] + 200)
df_combined2['APR timedelta Liq + 500'] = 100 * df_combined2['fees per year'] / (df_combined2['reserveUSD'] + 500)
df_combined2['APR timedelta Liq + 1000'] = 100 * df_combined2['fees per year'] / (df_combined2['reserveUSD'] + 1000)
df_combined2['APR timedelta Liq + 2000'] = 100 * df_combined2['fees per year'] / (df_combined2['reserveUSD'] + 2000)
df_combined2['APR timedelta Liq - 200'] = 100 * df_combined2['fees per year'] / (df_combined2['reserveUSD'] - 200)
df_combined2['APR timedelta Liq - 500'] = 100 * df_combined2['fees per year'] / (df_combined2['reserveUSD'] - 500)
df_combined2['APR timedelta Liq - 1000'] = 100 * df_combined2['fees per year'] / (df_combined2['reserveUSD'] - 1000)
df_combined2['APR timedelta Liq - 2000'] = 100 * df_combined2['fees per year'] / (df_combined2['reserveUSD'] - 2000)
df_combined2 = df_combined2[['APR timedelta',
                             'APR timedelta Liq + 200',
                             'APR timedelta Liq + 500',
                             'APR timedelta Liq + 1000',
                             'APR timedelta Liq + 2000',
                             'APR timedelta Liq - 200',
                             'APR timedelta Liq - 500',
                             'APR timedelta Liq - 1000',
                             'APR timedelta Liq - 2000',                              
                             'reserveUSD']]
df_combined2.head(2)

,APR timedelta,APR timedelta Liq + 200,APR timedelta Liq + 500,APR timedelta Liq + 1000,APR timedelta Liq + 2000,APR timedelta Liq - 200,APR timedelta Liq - 500,APR timedelta Liq - 1000,APR timedelta Liq - 2000,reserveUSD
0x0c787944946d22922c9f41c477cc539700d35bb2,4.209087,4.204594,4.197872,4.186717,4.164583,4.213589,4.220362,4.231697,4.254552,187156.260133
0x1013ba833071fd8aca8bf2ac83e5515c5fb9e76f,4.412490,4.407356,4.399678,4.386940,4.361685,4.417635,4.425376,4.438338,4.464492,171704.209767


In [45]:
df_final_honeyswap_matic = pd.concat([df_combined2, df_yesterday],
                                 axis = 1)[['Pair',
                                            'date',
                                            'APR timedelta',
                                            'APR timedelta Liq + 200',
                                            'APR timedelta Liq + 500',
                                            'APR timedelta Liq + 1000',                                            
                                            'APR timedelta Liq + 2000',
                                            'APR timedelta Liq - 200',
                                            'APR timedelta Liq - 500',
                                            'APR timedelta Liq - 1000',                                            
                                            'APR timedelta Liq - 2000',                                                 
                                            'APR yday',
                                            'APR yday Liq + 200',
                                            'APR yday Liq + 500',
                                            'APR yday Liq + 1000',
                                            'APR yday Liq + 2000',
                                            'APR yday Liq - 200',
                                            'APR yday Liq - 500',
                                            'APR yday Liq - 1000',
                                            'APR yday Liq - 2000',                                            
                                            'reserveUSD',
                                            'volumeUSD']].set_index('Pair')\
                       .sort_values(by = 'APR timedelta Liq + 2000', ascending = False)

df_final_honeyswap_matic.columns = ['date',
                              'APR timedelta',
                              'APR timedelta Liq + 200',
                              'APR timedelta Liq + 500',
                              'APR timedelta Liq + 1000',                                    
                              'APR timedelta Liq + 2000',
                              'APR timedelta Liq - 200',
                              'APR timedelta Liq - 500',
                              'APR timedelta Liq - 1000',                                    
                              'APR timedelta Liq - 2000',                                    
                              'APR yday',
                              'APR yday Liq + 200',
                              'APR yday Liq + 500',
                              'APR yday Liq + 1000',
                              'APR yday Liq + 2000',
                              'APR yday Liq - 200',
                              'APR yday Liq - 500',
                              'APR yday Liq - 1000',
                              'APR yday Liq - 2000',                                     
                              'Liquidity timedelta',
                              'Liquidity yday',
                              'Volume yday']

df_final_honeyswap_matic.head(1)

# df_final_honeyswap_matic_stylized = df_final_honeyswap_matic.dropna()\
#                                                 .style.format({'APR timedelta': "{:0<4,.2f}",
#                                                                'APR timedelta Liq + 100': "{:0<4,.2f}",
#                                                            'APR yday': "{:0<4,.2f}",
#                                                            'APR yday Liq + 100': "{:0<4,.2f}",
#                                                            'Liquidity timedelta': "{:0<4,.0f}",
#                                                            'Liquidity yday': "{:0<4,.0f}",
#                                                            'Volume yday': "{:0<4,.0f}"})

# df_final_honeyswap_matic_stylized

,date,APR timedelta,APR timedelta Liq + 200,APR timedelta Liq + 500,APR timedelta Liq + 1000,APR timedelta Liq + 2000,APR timedelta Liq - 200,APR timedelta Liq - 500,APR timedelta Liq - 1000,APR timedelta Liq - 2000,APR yday,APR yday Liq + 200,APR yday Liq + 500,APR yday Liq + 1000,APR yday Liq + 2000,APR yday Liq - 200,APR yday Liq - 500,APR yday Liq - 1000,APR yday Liq - 2000,Liquidity timedelta,Liquidity yday,Volume yday
Pair,,,,,,,,,,,,,,,,,,,,,,
[Honeyswap MATIC] USDC-WETH,2021-07-06,10.643101,10.635147,10.623238,10.603449,10.564091,10.651068,10.663039,10.683052,10.723303,13.124,13.114,13.099,13.074,13.025,13.134,13.149,13.174,13.225,267409.960573,263193.354972,37853.927753


## Balancer Polygon

In [46]:
# client_exe = client('https://api.thegraph.com/subgraphs/name/balancer-labs/balancer-polygon-v2')
# #https://thegraph.com/explorer/subgraph/balancer-labs/balancer-polygon-v2
# fee = 0.0025

In [47]:
# query = gql('''
#             query {
#                   balancers {
#                     poolCount
#                   }
#                 }
#             ''')

# response = client_exe.execute(query)
# number_of_pairs = int(response['balancers'][0]['poolCount'])
# print(number_of_pairs)

In [48]:
# # make a list of ids and pairs on balancer

# df = pd.DataFrame([])
# response_dict = {}
# response_list = []


# for i in range(number_of_pairs):
#     if i % 100 == 0:
#         querystring = '''
#               query {
#               pools(first: '''+str(100)+' skip: '+str(i)+'''){
#                       tokens{symbol}
#                       id
#                   }
#                 }
#                 '''
#         try:
#             response = client_exe.execute(gql(querystring))
#             df = df.append(pd.DataFrame.from_dict(response['pools']))
#             print(len(df))
#         except:
#             pass
    
# df['tokens'] = df['tokens'].astype(str).str.replace("{'symbol': ",'', regex= True)\
#                                            .replace("}",'', regex= True)\
#                                            .replace("]",'', regex= True)\
#                                            .replace('\[', '', regex = True)\
#                                            .replace("'",'', regex= True)\
#                                            .replace(" ",'', regex= True)

# number_different_tokens = len(df['tokens'].str.split(',', expand = True).columns)
# column_list = []

# for token_column in range(number_different_tokens):
#     column_list.append('Token' + str(token_column + 1))
    
# df[column_list] = df['tokens'].str.split(',', expand = True)
# df.replace()

In [49]:
# # make a list of all tokens on Balancer that are not in my interesting tokens

# df_tokens = pd.DataFrame([])

# for column in column_list:
#     df_tokenpart = df[[column]]
#     df_tokenpart = df_tokenpart.rename(columns = {column:'Token'})
#     df_tokens = df_tokens.append(df_tokenpart)
    
# all_balancer_tokens = df_tokens.drop_duplicates().dropna().sort_values(by = 'Token')['Token'].unique().tolist()
# tokens_to_avoid = set(all_balancer_tokens) - set(interesting_tokens)

In [50]:
# # filter out the pairs on balancer that contain tokens to avoid
# df_filtered = df.copy()

# for column in column_list:
#     df_filtered = df_filtered.loc[~df[column].isin(tokens_to_avoid)]
    
# df_filtered = df_filtered[['id', 'tokens']]
# df_filtered

In [51]:
# df_timedelta = pd.DataFrame([])
# lenid = len(df_filtered['id'])

# for count, id in enumerate(df_filtered['id']):
#     print(f'{count+1:2} / {lenid} pair: {id}')

#     querystring = '''
#                   query {
#       pools(where: {id:"''' + id + '''"}){
#         totalSwapVolume
#       }
#     }'''
#     response = client_exe.execute(gql(querystring))

#     df1 = pd.DataFrame.from_dict(response['pools'])
#     df1['id'] = id
#     df1['timestamp'] = datetime.now()
#     df1['volumeUSD'] = df1['totalSwapVolume'].astype(float)
#     df_timedelta = df_timedelta.append(df1)
    
# df_timedelta = df_timedelta[['id', 'timestamp', 'volumeUSD']]
# df_timedelta.head(1)

In [52]:
# df_old = pd.read_csv("Balancer export MATIC.csv",
#                      usecols = ['id', 'timestamp', 'volumeUSD'])
# df_old = df_old[['id', 'timestamp', 'volumeUSD']]
# df_old['timestamp'] = pd.to_datetime(df_old['timestamp'])
# df_old.columns = ['id_old', 'timestamp_old', 'volumeUSD_old']
# df_old.head(1)

In [53]:
# df_timedelta.to_csv("Balancer export MATIC.csv", index = False)

In [54]:
# df_combined = pd.concat([df_timedelta.set_index('id'), df_old.set_index('id_old')], axis = 1)
# df_combined['timedelta'] = (df_combined['timestamp'] - df_combined['timestamp_old']) / np.timedelta64(1, 'Y')
# df_combined['volumeUSD delta'] = df_combined['volumeUSD'] - df_combined['volumeUSD_old']
# df_combined['fees per year'] = df_combined['volumeUSD delta'] * fee / df_combined['timedelta']

# df_combined.head(2)

In [55]:
# querystring = """
#               query {
#                     poolSnapshots(orderBy: timestamp, orderDirection: desc){
#                       timestamp
#                     }
#                     }
#                 """

# response = client_exe.execute(gql(querystring))

# df_time = pd.DataFrame.from_dict(response['poolSnapshots'])\
#                       .drop_duplicates()\
#                       .sort_values(by = 'timestamp', ascending = False).head(2)

# df_time['timestamp'].values.tolist()

In [56]:
# {
#   pools{
#     id
#     totalSwapVolume
#     totalLiquidity
#   }
# }

In [57]:
# """
# {
# poolSnapshots(where: {id:"0x36128d5436d2d70cab39c9af9cce146c38554ff0000100000000000000000008-1625270400"} orderBy: timestamp, orderDirection: desc first: 2){
#   amounts
#   totalShares
#   swapVolume
#   swapFees
#   timestamp
# }
# }
# """

In [58]:
# """
# {
# poolSnapshots(where: {id:"0x36128d5436d2d70cab39c9af9cce146c38554ff0000100000000000000000008-1625270400"} orderBy: timestamp, orderDirection: desc first: 2){
#   totalShares
#   swapVolume
#   swapFees
#   timestamp
#   pool{historicalValues(orderBy: block, orderDirection: desc first: 2){
#     poolLiquidity
#     poolId {
#       id
#     }
#     block
#     id
#   }}
# }
# }
# """

In [59]:
# df_last_two_days = pd.DataFrame([])
# lenid = len(df_filtered['id'])

# for count, id in enumerate(df_filtered['id']):
#     print(f'{count+1:2} / {lenid} pair: {id}')
        
#     querystring = '''
#               query {
#   pairDayDatas(where: {pairAddress:"''' + id + '''"} orderBy: date, orderDirection: desc first: 2) {
#     date
#     dailyVolumeUSD
#     reserveUSD
#   }
# }'''
#     response = client('https://api.thegraph.com/subgraphs/name/1hive/honeyswap-polygon').execute(gql(querystring))
#     response
    
#     df1 = pd.DataFrame.from_dict(response['pairDayDatas'])
#     df1['id'] = id
#     df_last_two_days = df_last_two_days.append(df1)
    
# df_last_two_days = df_last_two_days.rename(columns = {'dailyVolumeUSD':'volumeUSD'})

## FTM settings

In [60]:
lending_tokens_cream = ['USDC', 'WFTM', 'DAI', 'ETH', 'BTC', 'LINK', 'SUSHI', 'YFI', 'SNX', 'BAND', 'AAVE', 'COVER', 'CREAM', 'HEGIC']
remaining_interesting_tokens = ['WFTM', 'FTM', 'USDC', 'DAI', 'ETH', 'fUSDT', 'FETH', 'WBTC', 'FRAX', 'FRAX', 'ibBTC']

interesting_tokens = list(set(lending_tokens_cream +\
                              remaining_interesting_tokens))

## SpiritSwap

In [61]:
query = gql('''
query {
  spiritswapFactories{
    pairCount
  }
}
''')

response = client('https://api.thegraph.com/subgraphs/name/layer3org/spiritswap-analytics').execute(query)
number_of_pairs = int(response['spiritswapFactories'][0]['pairCount'])
print(number_of_pairs)

711


In [62]:
try:
    df = pd.read_excel('pairdata_Spiritswap_FTM.xlsx')
except:
    df = pd.DataFrame([])
    response_dict = {}
    response_list = []


    for i in range(number_of_pairs):
        if i % 100 == 0:
            querystring = '''
                  query {
                  pairs(first: '''+str(100)+' skip: '+str(i)+''' where: {volumeUSD_gt:0}){
                          token0{symbol}
                          token1{symbol}
                          id
                      }
                    }
                    '''
            try:
                response = client('https://api.thegraph.com/subgraphs/name/layer3org/spiritswap-analytics').execute(gql(querystring))
                df = df.append(pd.DataFrame.from_dict(response['pairs']))
                print(len(df))
            except:
                pass

In [63]:
if df.columns.tolist() == ['id','Pair', 'Token 0', 'Token 1']:
    pass
else:
    df['token0'] = df['token0'].astype(str).str.replace("{'symbol': ",'', regex= True)\
                                               .replace("}",'', regex= True)\
                                               .replace("'",'', regex= True)
    df['token1'] = df['token1'].astype(str).str.replace("{'symbol': ",'', regex= True)\
                                               .replace("}",'', regex= True)\
                                               .replace("'",'', regex= True)
    df['Pair'] = df['token0'] + '-' + df['token1']
    df.columns = ['id','Token 0', 'Token 1', 'Pair']
    df = df[['id','Pair', 'Token 0', 'Token 1']]
    df.head(2)

In [64]:
criteria = df['Token 0'].isin(interesting_tokens) & \
           df['Token 1'].isin(interesting_tokens)

df_filtered = df.loc[criteria,:]
df_filtered.to_excel("pairdata_Spiritswap_FTM.xlsx", index = False)
df_filtered.head(2)

,id,Pair,Token 0,Token 1
0,0x040dd0d0f5e2a01feb0c5457abb588b23cf4c43a,WFTM-CREAM,WFTM,CREAM
1,0x0ec0e1629e776272fa3e55548d4a656be0eedcf4,WFTM-FRAX,WFTM,FRAX


In [65]:
df_timedelta = pd.DataFrame([])
lenid = len(df_filtered['id'])

for count, id in enumerate(df_filtered['id']):
    print(f'{count+1:2} / {lenid} pair: {id}')

    querystring = '''
                  query {
      pairs(where: {id:"''' + id + '''"}){
        volumeUSD
      }
    }'''
    response = client('https://api.thegraph.com/subgraphs/name/layer3org/spiritswap-analytics').execute(gql(querystring))

    df1 = pd.DataFrame.from_dict(response['pairs'])
    df1['id'] = id
    df1['timestamp'] = datetime.now()
    df1['volumeUSD'] = df1['volumeUSD'].astype(float)
    df_timedelta = df_timedelta.append(df1)
    
df_timedelta = df_timedelta[['id', 'timestamp', 'volumeUSD']]
df_timedelta.head(1)

 1 / 21 pair: 0x040dd0d0f5e2a01feb0c5457abb588b23cf4c43a
 2 / 21 pair: 0x0ec0e1629e776272fa3e55548d4a656be0eedcf4
 3 / 21 pair: 0x13bac72a9b0d081aa3926ee7f98d0e1b21dd7d73
 4 / 21 pair: 0x279b2c897737a50405ed2091694f225d83f2d3ba
 5 / 21 pair: 0x33068ad825fbeedb0e57e0466e379290b699f57c
 6 / 21 pair: 0x4556f6afa9b016a83052d459290573a5d83c8764
 7 / 21 pair: 0x4fc38a2735c7da1d71ccabf6dec235a7da4ec52c
 8 / 21 pair: 0x5427e7ab61145c41b1e58453121245daef3929c3
 9 / 21 pair: 0x613bf4e46b4817015c01c6bb31c7ae9edaadc26e
10 / 21 pair: 0x6a65a67da11b68cd141581483a9f7a0cb3c47bae
11 / 21 pair: 0x6d134d63417dc67f57dbc9fcaad4b4cbfab3af2f
12 / 21 pair: 0x6d87359b1e3fd852bd36250018e3328d1afd78e7
13 / 21 pair: 0x9606d683d03f012dda296ef0ae9261207c4a5847
14 / 21 pair: 0x9fe4c0ce5f533e96c2b72d852f190961ad5a7bb3
15 / 21 pair: 0xb67647130d0d51b5a0d53e3ae99ea24ca6100ebb
16 / 21 pair: 0xd061c6586670792331e14a80f3b3bb267189c681
17 / 21 pair: 0xd14dd3c56d9bc306322d4cea0e1c49e9ddf045d4
18 / 21 pair: 0xd3a0c65e5cefcee

,id,timestamp,volumeUSD
0,0x040dd0d0f5e2a01feb0c5457abb588b23cf4c43a,2021-07-07 08:04:41.204891,4.464935e+06


In [66]:
df_old = pd.read_csv("Spiritswap export FTM.csv",
                     usecols = ['id', 'timestamp', 'volumeUSD'])
df_old = df_old[['id', 'timestamp', 'volumeUSD']]
df_old['timestamp'] = pd.to_datetime(df_old['timestamp'])
df_old.columns = ['id_old', 'timestamp_old', 'volumeUSD_old']
df_old.head(1)

,id_old,timestamp_old,volumeUSD_old
0,0x040dd0d0f5e2a01feb0c5457abb588b23cf4c43a,2021-07-06 18:06:06.184899,4.459481e+06


In [67]:
df_timedelta.to_csv("Spiritswap export FTM.csv", index = False)

In [68]:
df_combined = pd.concat([df_timedelta.set_index('id'), df_old.set_index('id_old')], axis = 1)
df_combined['timedelta'] = (df_combined['timestamp'] - df_combined['timestamp_old']) / np.timedelta64(1, 'Y')
df_combined['volumeUSD delta'] = df_combined['volumeUSD'] - df_combined['volumeUSD_old']
df_combined['fees per year'] = df_combined['volumeUSD delta'] * 0.003 / df_combined['timedelta']

df_combined.head(2)

,timestamp,volumeUSD,timestamp_old,volumeUSD_old,timedelta,volumeUSD delta,fees per year
0x040dd0d0f5e2a01feb0c5457abb588b23cf4c43a,2021-07-07 08:04:41.204891,4.464935e+06,2021-07-06 18:06:06.184899,4.459481e+06,0.001594,5453.566651,10261.226632
0x0ec0e1629e776272fa3e55548d4a656be0eedcf4,2021-07-07 08:04:41.403679,1.064116e+07,2021-07-06 18:06:06.397529,1.063122e+07,0.001594,9940.052883,18702.836133


In [69]:
df_last_two_days = pd.DataFrame([])
lenid = len(df_filtered['id'])

for count, id in enumerate(df_filtered['id']):
    print(f'{count+1:2} / {lenid} pair: {id}')
        
    querystring = '''
              query {
  pairDayDatas(where: {pairAddress:"''' + id + '''"} orderBy: date, orderDirection: desc first: 2) {
    date
    dailyVolumeUSD
    reserveUSD
  }
}'''
    response = client('https://api.thegraph.com/subgraphs/name/layer3org/spiritswap-analytics').execute(gql(querystring))
    response
    
    df1 = pd.DataFrame.from_dict(response['pairDayDatas'])
    df1['id'] = id
    df_last_two_days = df_last_two_days.append(df1)
    
df_last_two_days = df_last_two_days.rename(columns = {'dailyVolumeUSD':'volumeUSD'})
df_last_two_days[['volumeUSD','reserveUSD']] = df_last_two_days[['volumeUSD','reserveUSD']].astype(float)
df_last_two_days['date'] = pd.to_datetime(df_last_two_days['date'], unit='s').dt.date
df_last_two_days.head(1)

 1 / 21 pair: 0x040dd0d0f5e2a01feb0c5457abb588b23cf4c43a
 2 / 21 pair: 0x0ec0e1629e776272fa3e55548d4a656be0eedcf4
 3 / 21 pair: 0x13bac72a9b0d081aa3926ee7f98d0e1b21dd7d73
 4 / 21 pair: 0x279b2c897737a50405ed2091694f225d83f2d3ba
 5 / 21 pair: 0x33068ad825fbeedb0e57e0466e379290b699f57c
 6 / 21 pair: 0x4556f6afa9b016a83052d459290573a5d83c8764
 7 / 21 pair: 0x4fc38a2735c7da1d71ccabf6dec235a7da4ec52c
 8 / 21 pair: 0x5427e7ab61145c41b1e58453121245daef3929c3
 9 / 21 pair: 0x613bf4e46b4817015c01c6bb31c7ae9edaadc26e
10 / 21 pair: 0x6a65a67da11b68cd141581483a9f7a0cb3c47bae
11 / 21 pair: 0x6d134d63417dc67f57dbc9fcaad4b4cbfab3af2f
12 / 21 pair: 0x6d87359b1e3fd852bd36250018e3328d1afd78e7
13 / 21 pair: 0x9606d683d03f012dda296ef0ae9261207c4a5847
14 / 21 pair: 0x9fe4c0ce5f533e96c2b72d852f190961ad5a7bb3
15 / 21 pair: 0xb67647130d0d51b5a0d53e3ae99ea24ca6100ebb
16 / 21 pair: 0xd061c6586670792331e14a80f3b3bb267189c681
17 / 21 pair: 0xd14dd3c56d9bc306322d4cea0e1c49e9ddf045d4
18 / 21 pair: 0xd3a0c65e5cefcee

,volumeUSD,date,reserveUSD,id
0,5318.450174,2021-07-07,282201.417029,0x040dd0d0f5e2a01feb0c5457abb588b23cf4c43a


In [70]:
df_last_two_days_incl_pairs = df_last_two_days.merge(df_filtered,
                                                     left_on = 'id',
                                                     right_on = 'id',
                                                     how = 'left',
                                                     validate = 'many_to_one')

df_last_two_days_incl_pairs['fee'] = round(df_last_two_days_incl_pairs['volumeUSD'] * 0.003, 6)
df_last_two_days_incl_pairs['APR yday'] = round(100 * df_last_two_days_incl_pairs['fee'] * 365 / df_last_two_days_incl_pairs['reserveUSD'], 3)
df_last_two_days_incl_pairs['Pair'] = '[SpiritSwap FTM] ' + df_last_two_days_incl_pairs['Pair']
df_last_two_days_incl_pairs['APR yday Liq + 200'] = round(100 * df_last_two_days_incl_pairs['fee'] * 365 / (df_last_two_days_incl_pairs['reserveUSD'] + 200), 3)
df_last_two_days_incl_pairs['APR yday Liq + 500'] = round(100 * df_last_two_days_incl_pairs['fee'] * 365 / (df_last_two_days_incl_pairs['reserveUSD'] + 500), 3)
df_last_two_days_incl_pairs['APR yday Liq + 1000'] = round(100 * df_last_two_days_incl_pairs['fee'] * 365 / (df_last_two_days_incl_pairs['reserveUSD'] + 1000), 3)
df_last_two_days_incl_pairs['APR yday Liq + 2000'] = round(100 * df_last_two_days_incl_pairs['fee'] * 365 / (df_last_two_days_incl_pairs['reserveUSD'] + 2000), 3)
df_last_two_days_incl_pairs['APR yday Liq - 200'] = round(100 * df_last_two_days_incl_pairs['fee'] * 365 / (df_last_two_days_incl_pairs['reserveUSD'] - 200), 3)
df_last_two_days_incl_pairs['APR yday Liq - 500'] = round(100 * df_last_two_days_incl_pairs['fee'] * 365 / (df_last_two_days_incl_pairs['reserveUSD'] - 500), 3)
df_last_two_days_incl_pairs['APR yday Liq - 1000'] = round(100 * df_last_two_days_incl_pairs['fee'] * 365 / (df_last_two_days_incl_pairs['reserveUSD'] - 1000), 3)
df_last_two_days_incl_pairs['APR yday Liq - 2000'] = round(100 * df_last_two_days_incl_pairs['fee'] * 365 / (df_last_two_days_incl_pairs['reserveUSD'] - 2000), 3)

df_last_two_days_incl_pairs.head(1)

,volumeUSD,date,reserveUSD,id,Pair,Token 0,Token 1,fee,APR yday,APR yday Liq + 200,APR yday Liq + 500,APR yday Liq + 1000,APR yday Liq + 2000,APR yday Liq - 200,APR yday Liq - 500,APR yday Liq - 1000,APR yday Liq - 2000
0,5318.450174,2021-07-07,282201.417029,0x040dd0d0f5e2a01feb0c5457abb588b23cf4c43a,[SpiritSwap FTM] WFTM-CREAM,WFTM,CREAM,15.955351,2.064,2.062,2.06,2.056,2.049,2.065,2.067,2.071,2.078


In [71]:
df_yesterday = df_last_two_days_incl_pairs.sort_values(by = 'date',
                                                       ascending = False)\
                                          .drop_duplicates(subset = ['Pair'], keep = 'last')\
                                          .reset_index(drop = True)\
                                          .drop(columns = ['Token 0', 'Token 1'])\
                                          .set_index('id')

df_yesterday.head(1)

,volumeUSD,date,reserveUSD,Pair,fee,APR yday,APR yday Liq + 200,APR yday Liq + 500,APR yday Liq + 1000,APR yday Liq + 2000,APR yday Liq - 200,APR yday Liq - 500,APR yday Liq - 1000,APR yday Liq - 2000
id,,,,,,,,,,,,,,
0xd52c57594f1b62cba87dbf079608332abe96e02e,189.28117,2021-07-06,4427.747653,[SpiritSwap FTM] USDC-FRAX,0.567844,4.681,4.479,4.206,3.819,3.225,4.902,5.277,6.047,8.537


In [72]:
# add in reserveUSD to calculate timedelta APYs

df_combined2 = pd.concat([df_combined,
                          df_last_two_days_incl_pairs.sort_values(by = 'date', ascending = False)\
                                                     .drop_duplicates(subset = ['Pair'],
                                                                      keep = 'first')\
                                                     .set_index('id')['reserveUSD']],
                         axis = 1)

df_combined2['APR timedelta'] = 100 * df_combined2['fees per year'] / df_combined2['reserveUSD']
df_combined2['APR timedelta Liq + 200'] = 100 * df_combined2['fees per year'] / (df_combined2['reserveUSD'] + 200)
df_combined2['APR timedelta Liq + 500'] = 100 * df_combined2['fees per year'] / (df_combined2['reserveUSD'] + 500)
df_combined2['APR timedelta Liq + 1000'] = 100 * df_combined2['fees per year'] / (df_combined2['reserveUSD'] + 1000)
df_combined2['APR timedelta Liq + 2000'] = 100 * df_combined2['fees per year'] / (df_combined2['reserveUSD'] + 2000)
df_combined2['APR timedelta Liq - 200'] = 100 * df_combined2['fees per year'] / (df_combined2['reserveUSD'] - 200)
df_combined2['APR timedelta Liq - 500'] = 100 * df_combined2['fees per year'] / (df_combined2['reserveUSD'] - 500)
df_combined2['APR timedelta Liq - 1000'] = 100 * df_combined2['fees per year'] / (df_combined2['reserveUSD'] - 1000)
df_combined2['APR timedelta Liq - 2000'] = 100 * df_combined2['fees per year'] / (df_combined2['reserveUSD'] - 2000)
df_combined2 = df_combined2[['APR timedelta',
                             'APR timedelta Liq + 200',
                             'APR timedelta Liq + 500',
                             'APR timedelta Liq + 1000',                             
                             'APR timedelta Liq + 2000',
                             'APR timedelta Liq - 200',
                             'APR timedelta Liq - 500',
                             'APR timedelta Liq - 1000',                             
                             'APR timedelta Liq - 2000',                             
                             'reserveUSD']]

df_combined2.head(1)

,APR timedelta,APR timedelta Liq + 200,APR timedelta Liq + 500,APR timedelta Liq + 1000,APR timedelta Liq + 2000,APR timedelta Liq - 200,APR timedelta Liq - 500,APR timedelta Liq - 1000,APR timedelta Liq - 2000,reserveUSD
0x040dd0d0f5e2a01feb0c5457abb588b23cf4c43a,3.636136,3.633561,3.629705,3.623296,3.610547,3.638715,3.64259,3.649066,3.662089,282201.417029


In [73]:
# df_final_spiritswap = pd.concat([df_combined2, df_yesterday],
#                                  axis = 1)[['Pair',
#                                             'date',
#                                             'APR timedelta',
#                                             'APR timedelta Liq + 200',
#                                             'APR timedelta Liq + 500',
#                                             'APR timedelta Liq + 1000',                                            
#                                             'APR timedelta Liq + 2000',
#                                             'APR timedelta Liq - 200',
#                                             'APR timedelta Liq - 500',
#                                             'APR timedelta Liq - 1000',                                            
#                                             'APR timedelta Liq - 2000',                                             
#                                             'APR yday',
#                                             'APR yday Liq + 200',
#                                             'APR yday Liq + 500',
#                                             'APR yday Liq + 1000',                                            
#                                             'APR yday Liq + 2000',
#                                             'APR yday Liq - 200',
#                                             'APR yday Liq - 500',
#                                             'APR yday Liq - 1000',                                            
#                                             'APR yday Liq - 2000',                                            
#                                             'reserveUSD',
#                                             'volumeUSD']].set_index('Pair')\
#                        .sort_values(by = 'APR timedelta Liq + 2000', ascending = False)

# df_final_spiritswap.columns = ['date',
#                               'APR timedelta',
#                                 'APR timedelta',
#                                 'APR timedelta Liq + 200',
#                                 'APR timedelta Liq + 500',
#                                 'APR timedelta Liq + 1000',                                            
#                                 'APR timedelta Liq + 2000',
#                                 'APR timedelta Liq - 200',
#                                 'APR timedelta Liq - 500',
#                                 'APR timedelta Liq - 1000',                                            
#                                 'APR timedelta Liq - 2000',     
#                               'APR yday',
#                                 'APR yday Liq + 200',
#                                 'APR yday Liq + 500',
#                                 'APR yday Liq + 1000',                                            
#                                 'APR yday Liq + 2000',
#                                 'APR yday Liq - 200',
#                                 'APR yday Liq - 500',
#                                 'APR yday Liq - 1000',                                            
#                                 'APR yday Liq - 2000', 
#                               'Liquidity timedelta',
#                               'Liquidity yday',
#                               'Volume yday']

# df_final_spiritswap.head(1)

df_final_spiritswap = pd.concat([df_combined2, df_yesterday],
                                 axis = 1)[['Pair',
                                            'date',
                                            'APR timedelta',
                                            'APR timedelta Liq + 200',
                                            'APR timedelta Liq + 500',
                                            'APR timedelta Liq + 1000',                                            
                                            'APR timedelta Liq + 2000',
                                            'APR timedelta Liq - 200',
                                            'APR timedelta Liq - 500',
                                            'APR timedelta Liq - 1000',                                            
                                            'APR timedelta Liq - 2000',                                                 
                                            'APR yday',
                                            'APR yday Liq + 200',
                                            'APR yday Liq + 500',
                                            'APR yday Liq + 1000',
                                            'APR yday Liq + 2000',
                                            'APR yday Liq - 200',
                                            'APR yday Liq - 500',
                                            'APR yday Liq - 1000',
                                            'APR yday Liq - 2000',                                            
                                            'reserveUSD',
                                            'volumeUSD']].set_index('Pair')\
                       .sort_values(by = 'APR timedelta Liq + 2000', ascending = False)

df_final_spiritswap.columns = ['date',
                              'APR timedelta',
                              'APR timedelta Liq + 200',
                              'APR timedelta Liq + 500',
                              'APR timedelta Liq + 1000',                                    
                              'APR timedelta Liq + 2000',
                              'APR timedelta Liq - 200',
                              'APR timedelta Liq - 500',
                              'APR timedelta Liq - 1000',                                    
                              'APR timedelta Liq - 2000',                                    
                              'APR yday',
                              'APR yday Liq + 200',
                              'APR yday Liq + 500',
                              'APR yday Liq + 1000',
                              'APR yday Liq + 2000',
                              'APR yday Liq - 200',
                              'APR yday Liq - 500',
                              'APR yday Liq - 1000',
                              'APR yday Liq - 2000',                                     
                              'Liquidity timedelta',
                              'Liquidity yday',
                              'Volume yday']

## SpookySwap

In [74]:
client_exe = client('https://api.thegraph.com/subgraphs/name/eerieeight/spooky-swap-exchange')
fee = 0.002

In [75]:
query = gql('''
            query {uniswapFactories {pairCount}}
            ''')

response = client_exe.execute(query)
number_of_pairs = int(response['uniswapFactories'][0]['pairCount'])
print(number_of_pairs)

498


In [76]:
# try:
#     df = pd.read_excel('pairdata_Spookyswap_FTM.xlsx')
# except:
df = pd.DataFrame([])
response_dict = {}
response_list = []

for i in range(number_of_pairs):
    if i % 100 == 0:
        querystring = '''
          query {
          pairs(first: '''+str(100)+' skip: '+str(i)+''' where: {volumeUSD_gt:0}){
                  token0 {symbol}
                  token1 {symbol}
                  id
              }
            }
            '''
        try:
            response = client_exe.execute(gql(querystring))
            df = df.append(pd.DataFrame.from_dict(response['pairs']))
            print(len(df))
        except:
            pass

100
200
259
259
259


In [77]:
if df.columns.tolist() == ['id','Pair', 'Token 0', 'Token 1']:
    pass
else:
    df['token0'] = df['token0'].astype(str).str.replace("{'symbol': ",'', regex= True)\
                                               .replace("}",'', regex= True)\
                                               .replace("'",'', regex= True)
    df['token1'] = df['token1'].astype(str).str.replace("{'symbol': ",'', regex= True)\
                                               .replace("}",'', regex= True)\
                                               .replace("'",'', regex= True)
    df['Pair'] = df['token0'] + '-' + df['token1']
    df.columns = ['id','Token 0', 'Token 1', 'Pair']
    df = df[['id','Pair', 'Token 0', 'Token 1']]
    criteria = df['Token 0'].isin(interesting_tokens) & \
               df['Token 1'].isin(interesting_tokens)

    df_filtered = df.loc[criteria,:]
df_filtered.head(1)

,id,Pair,Token 0,Token 1
4,0x06d173628be105fe81f1c82c9979ba79ebcafcb7,WFTM-SNX,WFTM,SNX


In [78]:
df_timedelta = pd.DataFrame([])
lenid = len(df_filtered['id'])

for count, id in enumerate(df_filtered['id']):
    print(f'{count+1:2} / {lenid} pair: {id}')

    querystring = '''
                  query {
      pairs(where: {id:"''' + id + '''"}){
        volumeUSD
      }
    }'''
    response = client_exe.execute(gql(querystring))

    df1 = pd.DataFrame.from_dict(response['pairs'])
    df1['id'] = id
    df1['timestamp'] = datetime.now()
    df1['volumeUSD'] = df1['volumeUSD'].astype(float)
    df_timedelta = df_timedelta.append(df1)
    
df_timedelta = df_timedelta[['id', 'timestamp', 'volumeUSD']]
df_timedelta.head(1)

 1 / 21 pair: 0x06d173628be105fe81f1c82c9979ba79ebcafcb7
 2 / 21 pair: 0x2b4c76d0dc16be1c31d4c1dc53bf9b45987fc75c
 3 / 21 pair: 0x484237bc35ca671302d19694c66d617142fbc235
 4 / 21 pair: 0x4a217fbf837c6115b183eaac0245d6a72e8a98e4
 5 / 21 pair: 0x5965e53aa80a0bcf1cd6dbdd72e6a9b2aa047410
 6 / 21 pair: 0x5dc7848bf215f1d99f2af3d2bf78fcdf238ee34b
 7 / 21 pair: 0x7fe139289e3837f8e6d352ecd013c6dbfd903a64
 8 / 21 pair: 0x89d9bc2f2d091cfbfc31e333d6dc555ddbc2fd29
 9 / 21 pair: 0x91b39d5584e2a7dc829f696235742cc293f2e8cf
10 / 21 pair: 0xa572bdf049382f1f98f9a430788dadd51a303969
11 / 21 pair: 0xb77b223490e1f5951ec79a8d09db9eab2adcb934
12 / 21 pair: 0xbf4d61d4cec3a9dff7452d8987e1cc2943e2eb4c
13 / 21 pair: 0xd92206379bd8203ac38225af006bb96bf1f12412
14 / 21 pair: 0xde1a99d2471470c33d9130c5dd485b0c99f3d0df
15 / 21 pair: 0xe120ffbda0d14f3bb6d6053e90e63c572a66a428
16 / 21 pair: 0xebf374bb21d83cf010cc7363918776adf6ff2bf6
17 / 21 pair: 0xec454eda10accdd66209c57af8c12924556f3abd
18 / 21 pair: 0xf0702249f4d3a25

,id,timestamp,volumeUSD
0,0x06d173628be105fe81f1c82c9979ba79ebcafcb7,2021-07-07 08:04:55.603729,4.773743e+06


In [79]:
df_old = pd.read_csv("Spookyswap export.csv",
                     usecols = ['id', 'timestamp', 'volumeUSD'])
df_old = df_old[['id', 'timestamp', 'volumeUSD']]
df_old['timestamp'] = pd.to_datetime(df_old['timestamp'])
df_old.columns = ['id_old', 'timestamp_old', 'volumeUSD_old']
df_old.head(1)

,id_old,timestamp_old,volumeUSD_old
0,0x06d173628be105fe81f1c82c9979ba79ebcafcb7,2021-07-06 18:06:21.279348,4.638848e+06


In [80]:
df_timedelta.to_csv("Spookyswap export.csv", index = False)

In [81]:
df_combined = pd.concat([df_timedelta.set_index('id'), df_old.set_index('id_old')], axis = 1)
df_combined['timedelta'] = (df_combined['timestamp'] - df_combined['timestamp_old']) / np.timedelta64(1, 'Y')
df_combined['volumeUSD delta'] = df_combined['volumeUSD'] - df_combined['volumeUSD_old']
df_combined['fees per year'] = df_combined['volumeUSD delta'] * fee / df_combined['timedelta']

df_combined.head(1)

,timestamp,volumeUSD,timestamp_old,volumeUSD_old,timedelta,volumeUSD delta,fees per year
0x06d173628be105fe81f1c82c9979ba79ebcafcb7,2021-07-07 08:04:55.603729,4.773743e+06,2021-07-06 18:06:21.279348,4.638848e+06,0.001594,134895.209982,169211.520528


In [82]:
df_last_two_days = pd.DataFrame([])
lenid = len(df_filtered['id'])

for count, id in enumerate(df_filtered['id']):
    print(f'{count+1:2} / {lenid} pair: {id}')
        
    querystring = '''
                  query {
                         pairDayDatas(where: {pairAddress:"''' + id + '''"}
                                      orderBy: date
                                      orderDirection: desc 
                                      first: 2) 
                                      {
                            date
                            dailyVolumeUSD
                            reserveUSD}
                        }
                  '''
    response = client_exe.execute(gql(querystring))
    
    df1 = pd.DataFrame.from_dict(response['pairDayDatas'])
    df1['id'] = id
    df_last_two_days = df_last_two_days.append(df1)
    
df_last_two_days = df_last_two_days.rename(columns = {'dailyVolumeUSD':'volumeUSD'})
df_last_two_days[['volumeUSD','reserveUSD']] = df_last_two_days[['volumeUSD','reserveUSD']].astype(float)
df_last_two_days['date'] = pd.to_datetime(df_last_two_days['date'], unit='s').dt.date
df_last_two_days.head(1)

 1 / 21 pair: 0x06d173628be105fe81f1c82c9979ba79ebcafcb7
 2 / 21 pair: 0x2b4c76d0dc16be1c31d4c1dc53bf9b45987fc75c
 3 / 21 pair: 0x484237bc35ca671302d19694c66d617142fbc235
 4 / 21 pair: 0x4a217fbf837c6115b183eaac0245d6a72e8a98e4
 5 / 21 pair: 0x5965e53aa80a0bcf1cd6dbdd72e6a9b2aa047410
 6 / 21 pair: 0x5dc7848bf215f1d99f2af3d2bf78fcdf238ee34b
 7 / 21 pair: 0x7fe139289e3837f8e6d352ecd013c6dbfd903a64
 8 / 21 pair: 0x89d9bc2f2d091cfbfc31e333d6dc555ddbc2fd29
 9 / 21 pair: 0x91b39d5584e2a7dc829f696235742cc293f2e8cf
10 / 21 pair: 0xa572bdf049382f1f98f9a430788dadd51a303969
11 / 21 pair: 0xb77b223490e1f5951ec79a8d09db9eab2adcb934
12 / 21 pair: 0xbf4d61d4cec3a9dff7452d8987e1cc2943e2eb4c
13 / 21 pair: 0xd92206379bd8203ac38225af006bb96bf1f12412
14 / 21 pair: 0xde1a99d2471470c33d9130c5dd485b0c99f3d0df
15 / 21 pair: 0xe120ffbda0d14f3bb6d6053e90e63c572a66a428
16 / 21 pair: 0xebf374bb21d83cf010cc7363918776adf6ff2bf6
17 / 21 pair: 0xec454eda10accdd66209c57af8c12924556f3abd
18 / 21 pair: 0xf0702249f4d3a25

,volumeUSD,date,reserveUSD,id
0,63963.342712,2021-07-07,1.224178e+06,0x06d173628be105fe81f1c82c9979ba79ebcafcb7


In [83]:
df_last_two_days_incl_pairs = df_last_two_days.merge(df_filtered,
                                                     left_on = 'id',
                                                     right_on = 'id',
                                                     how = 'left',
                                                     validate = 'many_to_one')

df_last_two_days_incl_pairs['fee'] = round(df_last_two_days_incl_pairs['volumeUSD'] * fee, 6)
df_last_two_days_incl_pairs['APR yday'] = round(100 * df_last_two_days_incl_pairs['fee'] * 365 / df_last_two_days_incl_pairs['reserveUSD'], 3)
df_last_two_days_incl_pairs['Pair'] = '[SpookySwap FTM] ' + df_last_two_days_incl_pairs['Pair']
df_last_two_days_incl_pairs['APR yday Liq + 200'] = round(100 * df_last_two_days_incl_pairs['fee'] * 365 / (df_last_two_days_incl_pairs['reserveUSD'] + 200), 3)
df_last_two_days_incl_pairs['APR yday Liq + 500'] = round(100 * df_last_two_days_incl_pairs['fee'] * 365 / (df_last_two_days_incl_pairs['reserveUSD'] + 500), 3)
df_last_two_days_incl_pairs['APR yday Liq + 1000'] = round(100 * df_last_two_days_incl_pairs['fee'] * 365 / (df_last_two_days_incl_pairs['reserveUSD'] + 1000), 3)
df_last_two_days_incl_pairs['APR yday Liq + 2000'] = round(100 * df_last_two_days_incl_pairs['fee'] * 365 / (df_last_two_days_incl_pairs['reserveUSD'] + 2000), 3)
df_last_two_days_incl_pairs['APR yday Liq - 200'] = round(100 * df_last_two_days_incl_pairs['fee'] * 365 / (df_last_two_days_incl_pairs['reserveUSD'] - 200), 3)
df_last_two_days_incl_pairs['APR yday Liq - 500'] = round(100 * df_last_two_days_incl_pairs['fee'] * 365 / (df_last_two_days_incl_pairs['reserveUSD'] - 500), 3)
df_last_two_days_incl_pairs['APR yday Liq - 1000'] = round(100 * df_last_two_days_incl_pairs['fee'] * 365 / (df_last_two_days_incl_pairs['reserveUSD'] - 1000), 3)
df_last_two_days_incl_pairs['APR yday Liq - 2000'] = round(100 * df_last_two_days_incl_pairs['fee'] * 365 / (df_last_two_days_incl_pairs['reserveUSD'] - 2000), 3)

df_last_two_days_incl_pairs.head(1)

,volumeUSD,date,reserveUSD,id,Pair,Token 0,Token 1,fee,APR yday,APR yday Liq + 200,APR yday Liq + 500,APR yday Liq + 1000,APR yday Liq + 2000,APR yday Liq - 200,APR yday Liq - 500,APR yday Liq - 1000,APR yday Liq - 2000
0,63963.342712,2021-07-07,1.224178e+06,0x06d173628be105fe81f1c82c9979ba79ebcafcb7,[SpookySwap FTM] WFTM-SNX,WFTM,SNX,127.926685,3.814,3.814,3.813,3.811,3.808,3.815,3.816,3.817,3.82


In [84]:
df_yesterday = df_last_two_days_incl_pairs.sort_values(by = 'date',
                                                       ascending = False)\
                                          .drop_duplicates(subset = ['Pair'], keep = 'last')\
                                          .reset_index(drop = True)\
                                          .drop(columns = ['Token 0', 'Token 1'])\
                                          .set_index('id')

df_yesterday.head(1)

,volumeUSD,date,reserveUSD,Pair,fee,APR yday,APR yday Liq + 200,APR yday Liq + 500,APR yday Liq + 1000,APR yday Liq + 2000,APR yday Liq - 200,APR yday Liq - 500,APR yday Liq - 1000,APR yday Liq - 2000
id,,,,,,,,,,,,,,
0xf0702249f4d3a25cd3ded7859a165693685ab577,319397.856782,2021-07-06,3.572898e+06,[SpookySwap FTM] WFTM-ETH,638.795714,6.526,6.525,6.525,6.524,6.522,6.526,6.527,6.528,6.529


In [85]:
# add in reserveUSD to calculate timedelta APYs

df_combined2 = pd.concat([df_combined,
                          df_last_two_days_incl_pairs.sort_values(by = 'date', ascending = False)\
                                                     .drop_duplicates(subset = ['Pair'],
                                                                      keep = 'first')\
                                                     .set_index('id')['reserveUSD']],
                         axis = 1)

df_combined2['APR timedelta'] = 100 * df_combined2['fees per year'] / df_combined2['reserveUSD']
df_combined2['APR timedelta Liq + 200'] = 100 * df_combined2['fees per year'] / (df_combined2['reserveUSD'] + 200)
df_combined2['APR timedelta Liq + 500'] = 100 * df_combined2['fees per year'] / (df_combined2['reserveUSD'] + 500)
df_combined2['APR timedelta Liq + 1000'] = 100 * df_combined2['fees per year'] / (df_combined2['reserveUSD'] + 1000)
df_combined2['APR timedelta Liq + 2000'] = 100 * df_combined2['fees per year'] / (df_combined2['reserveUSD'] + 2000)
df_combined2['APR timedelta Liq - 200'] = 100 * df_combined2['fees per year'] / (df_combined2['reserveUSD'] - 200)
df_combined2['APR timedelta Liq - 500'] = 100 * df_combined2['fees per year'] / (df_combined2['reserveUSD'] - 500)
df_combined2['APR timedelta Liq - 1000'] = 100 * df_combined2['fees per year'] / (df_combined2['reserveUSD'] - 1000)
df_combined2['APR timedelta Liq - 2000'] = 100 * df_combined2['fees per year'] / (df_combined2['reserveUSD'] - 2000)

df_combined2 = df_combined2[['APR timedelta',
                             'APR timedelta Liq + 200',
                             'APR timedelta Liq + 500',
                             'APR timedelta Liq + 1000',                             
                             'APR timedelta Liq + 2000',
                             'APR timedelta Liq - 200',
                             'APR timedelta Liq - 500',
                             'APR timedelta Liq - 1000',                             
                             'APR timedelta Liq - 2000',                             
                             'reserveUSD']]

df_combined2.head(1)

,APR timedelta,APR timedelta Liq + 200,APR timedelta Liq + 500,APR timedelta Liq + 1000,APR timedelta Liq + 2000,APR timedelta Liq - 200,APR timedelta Liq - 500,APR timedelta Liq - 1000,APR timedelta Liq - 2000,reserveUSD
0x06d173628be105fe81f1c82c9979ba79ebcafcb7,13.822464,13.820206,13.816821,13.811182,13.799918,13.824723,13.828112,13.833764,13.845083,1.224178e+06


In [86]:
df_final_spookyswap = pd.concat([df_combined2, df_yesterday],
                                 axis = 1)[['Pair',
                                            'date',
                                            'APR timedelta',
                                            'APR timedelta Liq + 200',
                                            'APR timedelta Liq + 500',
                                            'APR timedelta Liq + 1000',                                            
                                            'APR timedelta Liq + 2000',
                                            'APR timedelta Liq - 200',
                                            'APR timedelta Liq - 500',
                                            'APR timedelta Liq - 1000',                                            
                                            'APR timedelta Liq - 2000',                                            
                                            'APR yday',
                                            'APR yday Liq + 200',
                                            'APR yday Liq + 500',
                                            'APR yday Liq + 1000',                                            
                                            'APR yday Liq + 2000',
                                            'APR yday Liq - 200',
                                            'APR yday Liq - 500',
                                            'APR yday Liq - 1000',                                            
                                            'APR yday Liq - 2000',                                            
                                            'reserveUSD',
                                            'volumeUSD']].set_index('Pair')\
                       .sort_values(by = 'APR timedelta Liq + 2000', ascending = False)

df_final_spookyswap.columns = ['date',
                              'APR timedelta',
                              'APR timedelta Liq + 200',
                              'APR timedelta Liq + 500',
                              'APR timedelta Liq + 1000',                               
                              'APR timedelta Liq + 2000',
                              'APR timedelta Liq - 200',
                              'APR timedelta Liq - 500',
                              'APR timedelta Liq - 1000',                               
                              'APR timedelta Liq - 2000',                               
                              'APR yday',
                              'APR yday Liq + 200',
                              'APR yday Liq + 500',
                              'APR yday Liq + 1000',                               
                              'APR yday Liq + 2000',
                              'APR yday Liq - 200',
                              'APR yday Liq - 500',
                              'APR yday Liq - 1000',                               
                              'APR yday Liq - 2000',                               
                              'Liquidity timedelta',
                              'Liquidity yday',
                              'Volume yday']

df_final_spookyswap.head(1)

,date,APR timedelta,APR timedelta Liq + 200,APR timedelta Liq + 500,APR timedelta Liq + 1000,APR timedelta Liq + 2000,APR timedelta Liq - 200,APR timedelta Liq - 500,APR timedelta Liq - 1000,APR timedelta Liq - 2000,APR yday,APR yday Liq + 200,APR yday Liq + 500,APR yday Liq + 1000,APR yday Liq + 2000,APR yday Liq - 200,APR yday Liq - 500,APR yday Liq - 1000,APR yday Liq - 2000,Liquidity timedelta,Liquidity yday,Volume yday
Pair,,,,,,,,,,,,,,,,,,,,,,
[SpookySwap FTM] USDC-WFTM,2021-07-06,39.020427,39.018816,39.016399,39.012372,39.004319,39.022039,39.024456,39.028486,39.036548,34.883,34.882,34.88,34.876,34.869,34.885,34.887,34.891,34.898,4.842937e+06,4.749980e+06,2.269789e+06


## SushiSwap FTM

In [87]:
client_exe = client('https://api.thegraph.com/subgraphs/name/sushiswap/fantom-exchange')
fee = 0.003

In [88]:
query = gql('''
            query {factories {pairCount}}
            ''')

response = client_exe.execute(query)
number_of_pairs = int(response['factories'][0]['pairCount'])
print(number_of_pairs)

710


In [89]:
# try:
#     df = pd.read_excel('pairdata_Sushiswap_FTM.xlsx')
# except:
df = pd.DataFrame([])
response_dict = {}
response_list = []


for i in range(number_of_pairs):
    if i % 100 == 0:
        querystring = '''
              query {
              pairs(first: '''+str(100)+' skip: '+str(i)+''' where: {volumeUSD_gt:0}){
                      token0 {symbol}
                      token1 {symbol}
                      id
                  }
                }
                '''
        try:
            response = client_exe.execute(gql(querystring))
            df = df.append(pd.DataFrame.from_dict(response['pairs']))
            print(len(df))
        except:
            pass
            
df.head(1)

100
200
300
400
423
423
423
423


,id,token0,token1
0,0x0103715fd20a3f2e11fd7b3e646a5f6f6703d245,{'symbol': 'FETH'},{'symbol': 'FUSD'}


In [90]:
if df.columns.tolist() == ['id','Pair', 'Token 0', 'Token 1']:
    pass
else:
    df['token0'] = df['token0'].astype(str).str.replace("{'symbol': ",'', regex= True)\
                                               .replace("}",'', regex= True)\
                                               .replace("'",'', regex= True)
    df['token1'] = df['token1'].astype(str).str.replace("{'symbol': ",'', regex= True)\
                                               .replace("}",'', regex= True)\
                                               .replace("'",'', regex= True)
    df['Pair'] = df['token0'] + '-' + df['token1']
    df.columns = ['id','Token 0', 'Token 1', 'Pair']
    df = df[['id','Pair', 'Token 0', 'Token 1']]
    criteria = df['Token 0'].isin(interesting_tokens) & \
               df['Token 1'].isin(interesting_tokens)

    df_filtered = df.loc[criteria,:]
df_filtered.head(1)

,id,Pair,Token 0,Token 1
39,0x147e27be8e66f74c7659d88e2e6a13188873d58b,WFTM-CREAM,WFTM,CREAM


In [91]:
df_timedelta = pd.DataFrame([])
lenid = len(df_filtered['id'])

for count, id in enumerate(df_filtered['id']):
    print(f'{count+1:2} / {lenid} pair: {id}')

    querystring = '''
                  query {
      pairs(where: {id:"''' + id + '''"}){
        volumeUSD
      }
    }'''
    response = client_exe.execute(gql(querystring))

    df1 = pd.DataFrame.from_dict(response['pairs'])
    df1['id'] = id
    df1['timestamp'] = datetime.now()
    df1['volumeUSD'] = df1['volumeUSD'].astype(float)
    df_timedelta = df_timedelta.append(df1)
    
df_timedelta = df_timedelta[['id', 'timestamp', 'volumeUSD']]
df_timedelta.head(1)

 1 / 27 pair: 0x147e27be8e66f74c7659d88e2e6a13188873d58b
 2 / 27 pair: 0x1ca86e57103564f47ffcea7259a6ce8cc1301549
 3 / 27 pair: 0x27884c7647deb61c9f2b9202d1141bb84661756b
 4 / 27 pair: 0x280abc0d9727f4de3a46ca9cf93c16b4a9c127d1
 5 / 27 pair: 0x2d89bb3b1448152002453aa37fbe4051afdbf56d
 6 / 27 pair: 0x33c89f1f62e4847b6e8595904185594b324c4ba4
 7 / 27 pair: 0x33e29a9ebdd370a8d50656e822abfd3a910da1b6
 8 / 27 pair: 0x3d0bd54c48c2c433ea6fed609cc3d5fb7a77622b
 9 / 27 pair: 0x49d2e0dc99c7358d7a9a8633bf6df111d0ee7f65
10 / 27 pair: 0x52fdb10013bbe9dffe8f27f709a867cb9cc53c60
11 / 27 pair: 0x6aafe06996ac5b5d504ce7ba90dfdd5393679f0d
12 / 27 pair: 0x71c8bceece3daf9e27741d2cc1f03170f862555f
13 / 27 pair: 0x8be92f3d64e91d08ab1cc8a5c487da3f1695b11e
14 / 27 pair: 0x925e0fb729af31d6d387ca188297211a21f92dec
15 / 27 pair: 0xa48869049e36f8bfe0cc5cf655632626988c0140
16 / 27 pair: 0xafa1bf9b2a89dfd60499838c331538283b3346ed
17 / 27 pair: 0xbd0eec56da621b6b31e3d06614f7853624e1c0af
18 / 27 pair: 0xbeddbae6e24314a

,id,timestamp,volumeUSD
0,0x147e27be8e66f74c7659d88e2e6a13188873d58b,2021-07-07 08:05:09.365507,4.024066e+06


In [92]:
df_old = pd.read_csv("Sushiswap export FTM.csv",
                     usecols = ['id', 'timestamp', 'volumeUSD'])
df_old = df_old[['id', 'timestamp', 'volumeUSD']]
df_old['timestamp'] = pd.to_datetime(df_old['timestamp'])
df_old.columns = ['id_old', 'timestamp_old', 'volumeUSD_old']
df_old.head(1)

,id_old,timestamp_old,volumeUSD_old
0,0x147e27be8e66f74c7659d88e2e6a13188873d58b,2021-07-06 18:06:43.338163,4.020430e+06


In [93]:
df_timedelta.to_csv("Sushiswap export FTM.csv", index = False)

In [94]:
df_combined = pd.concat([df_timedelta.set_index('id'), df_old.set_index('id_old')], axis = 1)
df_combined['timedelta'] = (df_combined['timestamp'] - df_combined['timestamp_old']) / np.timedelta64(1, 'Y')
df_combined['volumeUSD delta'] = df_combined['volumeUSD'] - df_combined['volumeUSD_old']
df_combined['fees per year'] = df_combined['volumeUSD delta'] * fee / df_combined['timedelta']

df_combined.head(1)

,timestamp,volumeUSD,timestamp_old,volumeUSD_old,timedelta,volumeUSD delta,fees per year
0x147e27be8e66f74c7659d88e2e6a13188873d58b,2021-07-07 08:05:09.365507,4.024066e+06,2021-07-06 18:06:43.338163,4.020430e+06,0.001594,3636.000978,6842.586131


In [95]:
df_last_two_days = pd.DataFrame([])
lenid = len(df_filtered['id'])

for count, id in enumerate(df_filtered['id']):
    print(f'{count+1:2} / {lenid} pair: {id}')
        
    querystring = '''
                  query {
                         pairDayDatas(where: {pair:"''' + id + '''"}
                                      orderBy: date
                                      orderDirection: desc 
                                      first: 2) 
                                      {
                            date
                            volumeUSD
                            reserveUSD}
                        }
                  '''
    response = client_exe.execute(gql(querystring))
    
    df1 = pd.DataFrame.from_dict(response['pairDayDatas'])
    df1['id'] = id
    df_last_two_days = df_last_two_days.append(df1)
    
df_last_two_days[['volumeUSD','reserveUSD']] = df_last_two_days[['volumeUSD','reserveUSD']].astype(float)
df_last_two_days['date'] = pd.to_datetime(df_last_two_days['date'], unit='s').dt.date
df_last_two_days.head(1)

 1 / 27 pair: 0x147e27be8e66f74c7659d88e2e6a13188873d58b
 2 / 27 pair: 0x1ca86e57103564f47ffcea7259a6ce8cc1301549
 3 / 27 pair: 0x27884c7647deb61c9f2b9202d1141bb84661756b
 4 / 27 pair: 0x280abc0d9727f4de3a46ca9cf93c16b4a9c127d1
 5 / 27 pair: 0x2d89bb3b1448152002453aa37fbe4051afdbf56d
 6 / 27 pair: 0x33c89f1f62e4847b6e8595904185594b324c4ba4
 7 / 27 pair: 0x33e29a9ebdd370a8d50656e822abfd3a910da1b6
 8 / 27 pair: 0x3d0bd54c48c2c433ea6fed609cc3d5fb7a77622b
 9 / 27 pair: 0x49d2e0dc99c7358d7a9a8633bf6df111d0ee7f65
10 / 27 pair: 0x52fdb10013bbe9dffe8f27f709a867cb9cc53c60
11 / 27 pair: 0x6aafe06996ac5b5d504ce7ba90dfdd5393679f0d
12 / 27 pair: 0x71c8bceece3daf9e27741d2cc1f03170f862555f
13 / 27 pair: 0x8be92f3d64e91d08ab1cc8a5c487da3f1695b11e
14 / 27 pair: 0x925e0fb729af31d6d387ca188297211a21f92dec
15 / 27 pair: 0xa48869049e36f8bfe0cc5cf655632626988c0140
16 / 27 pair: 0xafa1bf9b2a89dfd60499838c331538283b3346ed
17 / 27 pair: 0xbd0eec56da621b6b31e3d06614f7853624e1c0af
18 / 27 pair: 0xbeddbae6e24314a

,date,reserveUSD,volumeUSD,id
0,2021-07-07,368834.658153,3267.446156,0x147e27be8e66f74c7659d88e2e6a13188873d58b


In [96]:
df_last_two_days_incl_pairs = df_last_two_days.merge(df_filtered,
                                                     left_on = 'id',
                                                     right_on = 'id',
                                                     how = 'left',
                                                     validate = 'many_to_one')

df_last_two_days_incl_pairs['fee'] = round(df_last_two_days_incl_pairs['volumeUSD'] * fee, 6)
df_last_two_days_incl_pairs['APR yday'] = round(100 * df_last_two_days_incl_pairs['fee'] * 365 / df_last_two_days_incl_pairs['reserveUSD'], 3)
df_last_two_days_incl_pairs['Pair'] = '[SushiSwap FTM] ' + df_last_two_days_incl_pairs['Pair']
df_last_two_days_incl_pairs['APR yday Liq + 200'] = round(100 * df_last_two_days_incl_pairs['fee'] * 365 / (df_last_two_days_incl_pairs['reserveUSD'] + 200), 3)
df_last_two_days_incl_pairs['APR yday Liq + 500'] = round(100 * df_last_two_days_incl_pairs['fee'] * 365 / (df_last_two_days_incl_pairs['reserveUSD'] + 500), 3)
df_last_two_days_incl_pairs['APR yday Liq + 1000'] = round(100 * df_last_two_days_incl_pairs['fee'] * 365 / (df_last_two_days_incl_pairs['reserveUSD'] + 1000), 3)
df_last_two_days_incl_pairs['APR yday Liq + 2000'] = round(100 * df_last_two_days_incl_pairs['fee'] * 365 / (df_last_two_days_incl_pairs['reserveUSD'] + 2000), 3)
df_last_two_days_incl_pairs['APR yday Liq - 200'] = round(100 * df_last_two_days_incl_pairs['fee'] * 365 / (df_last_two_days_incl_pairs['reserveUSD'] - 200), 3)
df_last_two_days_incl_pairs['APR yday Liq - 500'] = round(100 * df_last_two_days_incl_pairs['fee'] * 365 / (df_last_two_days_incl_pairs['reserveUSD'] - 500), 3)
df_last_two_days_incl_pairs['APR yday Liq - 1000'] = round(100 * df_last_two_days_incl_pairs['fee'] * 365 / (df_last_two_days_incl_pairs['reserveUSD'] - 1000), 3)
df_last_two_days_incl_pairs['APR yday Liq - 2000'] = round(100 * df_last_two_days_incl_pairs['fee'] * 365 / (df_last_two_days_incl_pairs['reserveUSD'] - 2000), 3)

df_last_two_days_incl_pairs.head(1)

,date,reserveUSD,volumeUSD,id,Pair,Token 0,Token 1,fee,APR yday,APR yday Liq + 200,APR yday Liq + 500,APR yday Liq + 1000,APR yday Liq + 2000,APR yday Liq - 200,APR yday Liq - 500,APR yday Liq - 1000,APR yday Liq - 2000
0,2021-07-07,368834.658153,3267.446156,0x147e27be8e66f74c7659d88e2e6a13188873d58b,[SushiSwap FTM] WFTM-CREAM,WFTM,CREAM,9.802338,0.97,0.97,0.969,0.967,0.965,0.971,0.971,0.973,0.975


In [97]:
df_yesterday = df_last_two_days_incl_pairs.sort_values(by = 'date',
                                                       ascending = False)\
                                          .drop_duplicates(subset = ['Pair'], keep = 'last')\
                                          .reset_index(drop = True)\
                                          .drop(columns = ['Token 0', 'Token 1'])\
                                          .set_index('id')

df_yesterday.head(1)

,date,reserveUSD,volumeUSD,Pair,fee,APR yday,APR yday Liq + 200,APR yday Liq + 500,APR yday Liq + 1000,APR yday Liq + 2000,APR yday Liq - 200,APR yday Liq - 500,APR yday Liq - 1000,APR yday Liq - 2000
id,,,,,,,,,,,,,,
0xd32f2eb49e91aa160946f3538564118388d6246a,2021-07-06,337712.775111,50194.909449,[SushiSwap FTM] WFTM-DAI,150.584728,16.275,16.266,16.251,16.227,16.179,16.285,16.299,16.324,16.372


In [98]:
# add in reserveUSD to calculate timedelta APYs

df_combined2 = pd.concat([df_combined,
                          df_last_two_days_incl_pairs.sort_values(by = 'date', ascending = False)\
                                                     .drop_duplicates(subset = ['Pair'],
                                                                      keep = 'first')\
                                                     .set_index('id')['reserveUSD']],
                         axis = 1)

df_combined2['APR timedelta'] = 100 * df_combined2['fees per year'] / df_combined2['reserveUSD']
df_combined2['APR timedelta Liq + 200'] = 100 * df_combined2['fees per year'] / (df_combined2['reserveUSD'] + 200)
df_combined2['APR timedelta Liq + 500'] = 100 * df_combined2['fees per year'] / (df_combined2['reserveUSD'] + 500)
df_combined2['APR timedelta Liq + 1000'] = 100 * df_combined2['fees per year'] / (df_combined2['reserveUSD'] + 1000)
df_combined2['APR timedelta Liq + 2000'] = 100 * df_combined2['fees per year'] / (df_combined2['reserveUSD'] + 2000)
df_combined2['APR timedelta Liq - 200'] = 100 * df_combined2['fees per year'] / (df_combined2['reserveUSD'] - 200)
df_combined2['APR timedelta Liq - 500'] = 100 * df_combined2['fees per year'] / (df_combined2['reserveUSD'] - 500)
df_combined2['APR timedelta Liq - 1000'] = 100 * df_combined2['fees per year'] / (df_combined2['reserveUSD'] - 1000)
df_combined2['APR timedelta Liq - 2000'] = 100 * df_combined2['fees per year'] / (df_combined2['reserveUSD'] - 2000)

df_combined2 = df_combined2[['APR timedelta',
                             'APR timedelta Liq + 200',
                             'APR timedelta Liq + 500',
                             'APR timedelta Liq + 1000',                             
                             'APR timedelta Liq + 2000',
                             'APR timedelta Liq - 200',
                             'APR timedelta Liq - 500',
                             'APR timedelta Liq - 1000',                             
                             'APR timedelta Liq - 2000',                             
                             'reserveUSD']]

df_combined2.head(1)

,APR timedelta,APR timedelta Liq + 200,APR timedelta Liq + 500,APR timedelta Liq + 1000,APR timedelta Liq + 2000,APR timedelta Liq - 200,APR timedelta Liq - 500,APR timedelta Liq - 1000,APR timedelta Liq - 2000,reserveUSD
0x147e27be8e66f74c7659d88e2e6a13188873d58b,1.855191,1.854185,1.852679,1.850174,1.845185,1.856197,1.857709,1.860234,1.865305,368834.658153


In [99]:
df_final_sushiswap_ftm = pd.concat([df_combined2, df_yesterday],
                                 axis = 1)[['Pair',
                                            'date',
                                            'APR timedelta',
                                            'APR timedelta Liq + 200',
                                            'APR timedelta Liq + 500',
                                            'APR timedelta Liq + 1000',                                            
                                            'APR timedelta Liq + 2000',
                                            'APR timedelta Liq - 200',
                                            'APR timedelta Liq - 500',
                                            'APR timedelta Liq - 1000',                                            
                                            'APR timedelta Liq - 2000',                                            
                                            'APR yday',
                                            'APR yday Liq + 200',
                                            'APR yday Liq + 500',
                                            'APR yday Liq + 1000',                                            
                                            'APR yday Liq + 2000',
                                            'APR yday Liq - 200',
                                            'APR yday Liq - 500',
                                            'APR yday Liq - 1000',                                            
                                            'APR yday Liq - 2000',                                            
                                            'reserveUSD',
                                            'volumeUSD']].set_index('Pair')\
                       .sort_values(by = 'APR timedelta Liq + 2000', ascending = False)

df_final_sushiswap_ftm.columns = ['date',
                              'APR timedelta',
                                'APR timedelta Liq + 200',
                                    'APR timedelta Liq + 500',
                                    'APR timedelta Liq + 1000',                                            
                                    'APR timedelta Liq + 2000',
                                    'APR timedelta Liq - 200',
                                    'APR timedelta Liq - 500',
                                    'APR timedelta Liq - 1000',                                            
                                    'APR timedelta Liq - 2000',   
                              'APR yday',
                                'APR yday Liq + 200',
                                    'APR yday Liq + 500',
                                    'APR yday Liq + 1000',                                            
                                    'APR yday Liq + 2000',
                                    'APR yday Liq - 200',
                                    'APR yday Liq - 500',
                                    'APR yday Liq - 1000',                                            
                                    'APR yday Liq - 2000',  
                              'Liquidity timedelta',
                              'Liquidity yday',
                              'Volume yday']

df_final_sushiswap_ftm.head(1)

,date,APR timedelta,APR timedelta Liq + 200,APR timedelta Liq + 500,APR timedelta Liq + 1000,APR timedelta Liq + 2000,APR timedelta Liq - 200,APR timedelta Liq - 500,APR timedelta Liq - 1000,APR timedelta Liq - 2000,APR yday,APR yday Liq + 200,APR yday Liq + 500,APR yday Liq + 1000,APR yday Liq + 2000,APR yday Liq - 200,APR yday Liq - 500,APR yday Liq - 1000,APR yday Liq - 2000,Liquidity timedelta,Liquidity yday,Volume yday
Pair,,,,,,,,,,,,,,,,,,,,,,
[SushiSwap FTM] USDC-WFTM,2021-07-06,40.094766,40.049237,39.981139,39.868154,39.64409,40.140397,40.20904,40.323967,40.555805,34.095,34.057,33.999,33.905,33.716,34.133,34.191,34.287,34.482,175932.03077,178106.400864,55456.790921


## WakaSwap FTM

In [102]:
client_exe = client('https://api.thegraph.com/subgraphs/name/waka-finance/waka-graph') # https://thegraph.com/explorer/subgraph/waka-finance/waka-graph
fee = 0.003

query = gql('''
            query {wakaSwapFactories {pairCount}}
            ''')

response = client_exe.execute(query)
number_of_pairs = int(response['wakaSwapFactories'][0]['pairCount'])
print(number_of_pairs)

# try:
#     df = pd.read_excel('pairdata_Sushiswap_FTM.xlsx')
# except:
df = pd.DataFrame([])
response_dict = {}
response_list = []


for i in range(number_of_pairs):
    if i % 100 == 0:
        querystring = '''
              query {
              pairs(first: '''+str(100)+' skip: '+str(i)+''' where: {volumeUSD_gt:0}){
                      token0 {symbol}
                      token1 {symbol}
                      id
                  }
                }
                '''
        try:
            response = client_exe.execute(gql(querystring))
            df = df.append(pd.DataFrame.from_dict(response['pairs']))
            print(len(df))
        except:
            pass
            
df.head(1)

if df.columns.tolist() == ['id','Pair', 'Token 0', 'Token 1']:
    pass
else:
    df['token0'] = df['token0'].astype(str).str.replace("{'symbol': ",'', regex= True)\
                                               .replace("}",'', regex= True)\
                                               .replace("'",'', regex= True)
    df['token1'] = df['token1'].astype(str).str.replace("{'symbol': ",'', regex= True)\
                                               .replace("}",'', regex= True)\
                                               .replace("'",'', regex= True)
    df['Pair'] = df['token0'] + '-' + df['token1']
    df.columns = ['id','Token 0', 'Token 1', 'Pair']
    df = df[['id','Pair', 'Token 0', 'Token 1']]
    criteria = df['Token 0'].isin(interesting_tokens) & \
               df['Token 1'].isin(interesting_tokens)

    df_filtered = df.loc[criteria,:]
df_filtered.head(1)

df_timedelta = pd.DataFrame([])
lenid = len(df_filtered['id'])

for count, id in enumerate(df_filtered['id']):
    print(f'{count+1:2} / {lenid} pair: {id}')

    querystring = '''
                  query {
      pairs(where: {id:"''' + id + '''"}){
        volumeUSD
      }
    }'''
    response = client_exe.execute(gql(querystring))

    df1 = pd.DataFrame.from_dict(response['pairs'])
    df1['id'] = id
    df1['timestamp'] = datetime.now()
    df1['volumeUSD'] = df1['volumeUSD'].astype(float)
    df_timedelta = df_timedelta.append(df1)
    
df_timedelta = df_timedelta[['id', 'timestamp', 'volumeUSD']]
df_timedelta.head(1)

35
23
 1 / 13 pair: 0x0eea1bf6342c71171882c2ac3838a9c78e00b5cb
 2 / 13 pair: 0x195c45044b121d8caf4c9e2d8f47fb81f4e12f21
 3 / 13 pair: 0x1b084cda6e11347b3c15eae537cf8a7ce39f96da
 4 / 13 pair: 0x2e783ce36830a6274761cc4a9d04d275553f3bc1
 5 / 13 pair: 0x32736ae178d0debf137326a0c0c6d50e374d9f19
 6 / 13 pair: 0x3c4ad49f26b47f00f7e5165b942d41367dd60a04
 7 / 13 pair: 0x60ff8c1dffc8252cfd94c07f65feb4ce90ee9b99
 8 / 13 pair: 0x68a242300e9547263aa8d57c1e4a7d75cf3c8e5f
 9 / 13 pair: 0x78da2bd2c18837c9d1b9008c81078e378f1952c6
10 / 13 pair: 0x97a490e7c90b4d8f6d18752672f4c8e3c35891a2
11 / 13 pair: 0x9c58df352fcd984700c57324a116845378adc5ba
12 / 13 pair: 0xb0e5d65b4c22ff9f099e449df03acb3a57bb5f27
13 / 13 pair: 0xff09b99aed8fb58c209904f6adc4a1fcceea79ac


,id,timestamp,volumeUSD
0,0x0eea1bf6342c71171882c2ac3838a9c78e00b5cb,2021-07-07 08:38:09.534470,7.202989e+06


In [103]:
df_timedelta.to_csv("WakaSwap export FTM.csv", index = False)

In [108]:
df_old = pd.read_csv("WakaSwap export FTM.csv",
                     usecols = ['id', 'timestamp', 'volumeUSD'])
df_old = df_old[['id', 'timestamp', 'volumeUSD']]
df_old['timestamp'] = pd.to_datetime(df_old['timestamp'])
df_old.columns = ['id_old', 'timestamp_old', 'volumeUSD_old']
df_old.head(1)

df_timedelta.to_csv("WakaSwap export FTM.csv", index = False)

df_combined = pd.concat([df_timedelta.set_index('id'), df_old.set_index('id_old')], axis = 1)
df_combined['timedelta'] = (df_combined['timestamp'] - df_combined['timestamp_old']) / np.timedelta64(1, 'Y')
df_combined['volumeUSD delta'] = df_combined['volumeUSD'] - df_combined['volumeUSD_old']
df_combined['fees per year'] = df_combined['volumeUSD delta'] * fee / df_combined['timedelta']

df_combined.head(1)

df_last_two_days = pd.DataFrame([])
lenid = len(df_filtered['id'])

for count, id in enumerate(df_filtered['id']):
    print(f'{count+1:2} / {lenid} pair: {id}')
        
    querystring = '''
                  query {
                         pairDayDatas(where: {pairAddress:"''' + id + '''"}
                                      orderBy: date
                                      orderDirection: desc 
                                      first: 2) 
                                      {
                            date
                            dailyVolumeUSD
                            reserveUSD}
                        }
                  '''
    response = client_exe.execute(gql(querystring))
    
    df1 = pd.DataFrame.from_dict(response['pairDayDatas'])
    df1['id'] = id
    df_last_two_days = df_last_two_days.append(df1)
    
df_last_two_days = df_last_two_days.rename(columns = {'dailyVolumeUSD':'volumeUSD'})
df_last_two_days[['volumeUSD','reserveUSD']] = df_last_two_days[['volumeUSD','reserveUSD']].astype(float)
df_last_two_days['date'] = pd.to_datetime(df_last_two_days['date'], unit='s').dt.date
df_last_two_days.head(1)

 1 / 13 pair: 0x0eea1bf6342c71171882c2ac3838a9c78e00b5cb
 2 / 13 pair: 0x195c45044b121d8caf4c9e2d8f47fb81f4e12f21
 3 / 13 pair: 0x1b084cda6e11347b3c15eae537cf8a7ce39f96da
 4 / 13 pair: 0x2e783ce36830a6274761cc4a9d04d275553f3bc1
 5 / 13 pair: 0x32736ae178d0debf137326a0c0c6d50e374d9f19
 6 / 13 pair: 0x3c4ad49f26b47f00f7e5165b942d41367dd60a04
 7 / 13 pair: 0x60ff8c1dffc8252cfd94c07f65feb4ce90ee9b99
 8 / 13 pair: 0x68a242300e9547263aa8d57c1e4a7d75cf3c8e5f
 9 / 13 pair: 0x78da2bd2c18837c9d1b9008c81078e378f1952c6
10 / 13 pair: 0x97a490e7c90b4d8f6d18752672f4c8e3c35891a2
11 / 13 pair: 0x9c58df352fcd984700c57324a116845378adc5ba
12 / 13 pair: 0xb0e5d65b4c22ff9f099e449df03acb3a57bb5f27
13 / 13 pair: 0xff09b99aed8fb58c209904f6adc4a1fcceea79ac


,volumeUSD,date,reserveUSD,id
0,3069.344429,2021-07-07,171826.99111,0x0eea1bf6342c71171882c2ac3838a9c78e00b5cb


In [109]:
df_last_two_days_incl_pairs = df_last_two_days.merge(df_filtered,
                                                     left_on = 'id',
                                                     right_on = 'id',
                                                     how = 'left',
                                                     validate = 'many_to_one')

df_last_two_days_incl_pairs['fee'] = round(df_last_two_days_incl_pairs['volumeUSD'] * fee, 6)
df_last_two_days_incl_pairs['APR yday'] = round(100 * df_last_two_days_incl_pairs['fee'] * 365 / df_last_two_days_incl_pairs['reserveUSD'], 3)
df_last_two_days_incl_pairs['Pair'] = '[WakaSwap FTM] ' + df_last_two_days_incl_pairs['Pair']
df_last_two_days_incl_pairs['APR yday Liq + 200'] = round(100 * df_last_two_days_incl_pairs['fee'] * 365 / (df_last_two_days_incl_pairs['reserveUSD'] + 200), 3)
df_last_two_days_incl_pairs['APR yday Liq + 500'] = round(100 * df_last_two_days_incl_pairs['fee'] * 365 / (df_last_two_days_incl_pairs['reserveUSD'] + 500), 3)
df_last_two_days_incl_pairs['APR yday Liq + 1000'] = round(100 * df_last_two_days_incl_pairs['fee'] * 365 / (df_last_two_days_incl_pairs['reserveUSD'] + 1000), 3)
df_last_two_days_incl_pairs['APR yday Liq + 2000'] = round(100 * df_last_two_days_incl_pairs['fee'] * 365 / (df_last_two_days_incl_pairs['reserveUSD'] + 2000), 3)
df_last_two_days_incl_pairs['APR yday Liq - 200'] = round(100 * df_last_two_days_incl_pairs['fee'] * 365 / (df_last_two_days_incl_pairs['reserveUSD'] - 200), 3)
df_last_two_days_incl_pairs['APR yday Liq - 500'] = round(100 * df_last_two_days_incl_pairs['fee'] * 365 / (df_last_two_days_incl_pairs['reserveUSD'] - 500), 3)
df_last_two_days_incl_pairs['APR yday Liq - 1000'] = round(100 * df_last_two_days_incl_pairs['fee'] * 365 / (df_last_two_days_incl_pairs['reserveUSD'] - 1000), 3)
df_last_two_days_incl_pairs['APR yday Liq - 2000'] = round(100 * df_last_two_days_incl_pairs['fee'] * 365 / (df_last_two_days_incl_pairs['reserveUSD'] - 2000), 3)

df_last_two_days_incl_pairs.head(1)

df_yesterday = df_last_two_days_incl_pairs.sort_values(by = 'date',
                                                       ascending = False)\
                                          .drop_duplicates(subset = ['Pair'], keep = 'last')\
                                          .reset_index(drop = True)\
                                          .drop(columns = ['Token 0', 'Token 1'])\
                                          .set_index('id')

df_yesterday.head(1)

# add in reserveUSD to calculate timedelta APYs

df_combined2 = pd.concat([df_combined,
                          df_last_two_days_incl_pairs.sort_values(by = 'date', ascending = False)\
                                                     .drop_duplicates(subset = ['Pair'],
                                                                      keep = 'first')\
                                                     .set_index('id')['reserveUSD']],
                         axis = 1)

df_combined2['APR timedelta'] = 100 * df_combined2['fees per year'] / df_combined2['reserveUSD']
df_combined2['APR timedelta Liq + 200'] = 100 * df_combined2['fees per year'] / (df_combined2['reserveUSD'] + 200)
df_combined2['APR timedelta Liq + 500'] = 100 * df_combined2['fees per year'] / (df_combined2['reserveUSD'] + 500)
df_combined2['APR timedelta Liq + 1000'] = 100 * df_combined2['fees per year'] / (df_combined2['reserveUSD'] + 1000)
df_combined2['APR timedelta Liq + 2000'] = 100 * df_combined2['fees per year'] / (df_combined2['reserveUSD'] + 2000)
df_combined2['APR timedelta Liq - 200'] = 100 * df_combined2['fees per year'] / (df_combined2['reserveUSD'] - 200)
df_combined2['APR timedelta Liq - 500'] = 100 * df_combined2['fees per year'] / (df_combined2['reserveUSD'] - 500)
df_combined2['APR timedelta Liq - 1000'] = 100 * df_combined2['fees per year'] / (df_combined2['reserveUSD'] - 1000)
df_combined2['APR timedelta Liq - 2000'] = 100 * df_combined2['fees per year'] / (df_combined2['reserveUSD'] - 2000)

df_combined2 = df_combined2[['APR timedelta',
                             'APR timedelta Liq + 200',
                             'APR timedelta Liq + 500',
                             'APR timedelta Liq + 1000',                             
                             'APR timedelta Liq + 2000',
                             'APR timedelta Liq - 200',
                             'APR timedelta Liq - 500',
                             'APR timedelta Liq - 1000',                             
                             'APR timedelta Liq - 2000',                             
                             'reserveUSD']]

df_combined2.head(1)

df_final_wakaswap_ftm = pd.concat([df_combined2, df_yesterday],
                                 axis = 1)[['Pair',
                                            'date',
                                            'APR timedelta',
                                            'APR timedelta Liq + 200',
                                            'APR timedelta Liq + 500',
                                            'APR timedelta Liq + 1000',                                            
                                            'APR timedelta Liq + 2000',
                                            'APR timedelta Liq - 200',
                                            'APR timedelta Liq - 500',
                                            'APR timedelta Liq - 1000',                                            
                                            'APR timedelta Liq - 2000',                                            
                                            'APR yday',
                                            'APR yday Liq + 200',
                                            'APR yday Liq + 500',
                                            'APR yday Liq + 1000',                                            
                                            'APR yday Liq + 2000',
                                            'APR yday Liq - 200',
                                            'APR yday Liq - 500',
                                            'APR yday Liq - 1000',                                            
                                            'APR yday Liq - 2000',                                            
                                            'reserveUSD',
                                            'volumeUSD']].set_index('Pair')\
                       .sort_values(by = 'APR timedelta Liq + 2000', ascending = False)

df_final_wakaswap_ftm.columns = ['date',
                              'APR timedelta',
                                'APR timedelta Liq + 200',
                                    'APR timedelta Liq + 500',
                                    'APR timedelta Liq + 1000',                                            
                                    'APR timedelta Liq + 2000',
                                    'APR timedelta Liq - 200',
                                    'APR timedelta Liq - 500',
                                    'APR timedelta Liq - 1000',                                            
                                    'APR timedelta Liq - 2000',   
                              'APR yday',
                                'APR yday Liq + 200',
                                    'APR yday Liq + 500',
                                    'APR yday Liq + 1000',                                            
                                    'APR yday Liq + 2000',
                                    'APR yday Liq - 200',
                                    'APR yday Liq - 500',
                                    'APR yday Liq - 1000',                                            
                                    'APR yday Liq - 2000',  
                              'Liquidity timedelta',
                              'Liquidity yday',
                              'Volume yday']

df_final_wakaswap_ftm.head(1)

,date,APR timedelta,APR timedelta Liq + 200,APR timedelta Liq + 500,APR timedelta Liq + 1000,APR timedelta Liq + 2000,APR timedelta Liq - 200,APR timedelta Liq - 500,APR timedelta Liq - 1000,APR timedelta Liq - 2000,APR yday,APR yday Liq + 200,APR yday Liq + 500,APR yday Liq + 1000,APR yday Liq + 2000,APR yday Liq - 200,APR yday Liq - 500,APR yday Liq - 1000,APR yday Liq - 2000,Liquidity timedelta,Liquidity yday,Volume yday
Pair,,,,,,,,,,,,,,,,,,,,,,
[WakaSwap FTM] WFTM-ETH,2021-07-06,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.716,2.712,2.708,2.699,2.683,2.719,2.724,2.732,2.749,171826.99111,165233.137094,4098.048741


In [100]:
pd.set_option('display.max_columns', None)

df_final_combined = df_final_sushiswap_matic.append(df_final_quickswap)\
                                            .append(df_final_honeyswap_matic)\
                                            .append(df_final_wakaswap_ftm)\
                                            .append(df_final_spiritswap)\
                                            .append(df_final_spookyswap)\
                                            .append(df_final_sushiswap_ftm)\
                                            .sort_values(by = 'APR timedelta Liq + 200',
                                                         ascending = False)

df_final_combined[['APR timedelta',
                   'APR timedelta Liq + 200',
                   'APR timedelta Liq + 500',
                   'APR timedelta Liq + 1000',                   
                   'APR timedelta Liq + 2000',
                   'APR timedelta Liq - 200',
                   'APR timedelta Liq - 500',
                   'APR timedelta Liq - 1000',                   
                   'APR timedelta Liq - 2000',                   
                   'APR yday',
                   'APR yday Liq + 200',
                   'APR yday Liq + 500',
                   'APR yday Liq + 1000',                   
                   'APR yday Liq + 2000',
                   'APR yday Liq - 200',
                   'APR yday Liq - 500',
                   'APR yday Liq - 1000',                   
                   'APR yday Liq - 2000']] = df_final_combined[['APR timedelta',
                                                                'APR timedelta Liq + 200',
                                                               'APR timedelta Liq + 500',
                                                               'APR timedelta Liq + 1000',                   
                                                               'APR timedelta Liq + 2000',
                                                               'APR timedelta Liq - 200',
                                                               'APR timedelta Liq - 500',
                                                               'APR timedelta Liq - 1000',                   
                                                               'APR timedelta Liq - 2000',                       
                                                                'APR yday',
                                                                'APR yday Liq + 200',
                                                               'APR yday Liq + 500',
                                                               'APR yday Liq + 1000',                   
                                                               'APR yday Liq + 2000',
                                                               'APR yday Liq - 200',
                                                               'APR yday Liq - 500',
                                                               'APR yday Liq - 1000',                   
                                                               'APR yday Liq - 2000']].round(2)

df_final_combined[['Liquidity timedelta',
                   'Liquidity yday',
                   'Volume yday']] = df_final_combined[['Liquidity timedelta',
                                                        'Liquidity yday',
                                                        'Volume yday']].fillna(0).astype(int)

criteria = (df_final_combined['Volume yday'] > 0) & (df_final_combined['APR timedelta'] > 0)

df_final_combined.loc[criteria, :].dropna(subset = ['date'])

,date,APR timedelta,APR timedelta Liq + 200,APR timedelta Liq + 500,APR timedelta Liq + 1000,APR timedelta Liq + 2000,APR timedelta Liq - 200,APR timedelta Liq - 500,APR timedelta Liq - 1000,APR timedelta Liq - 2000,APR yday,APR yday Liq + 200,APR yday Liq + 500,APR yday Liq + 1000,APR yday Liq + 2000,APR yday Liq - 200,APR yday Liq - 500,APR yday Liq - 1000,APR yday Liq - 2000,Liquidity timedelta,Liquidity yday,Volume yday
Pair,,,,,,,,,,,,,,,,,,,,,,
[Sushiswap MATIC] WMATIC-QUICK,2021-07-06,138.98,131.21,121.06,107.23,87.29,147.73,163.13,197.43,340.76,142.29,134.04,123.31,108.80,88.07,151.62,168.17,205.56,370.18,3377,3248,4221
[SpiritSwap FTM] WFTM-DAI,2021-07-06,98.82,74.18,53.99,37.14,22.87,147.98,582.77,-149.54,-42.56,92.02,68.58,49.62,33.97,20.83,139.80,631.98,-129.83,-38.06,602,585,491
[Sushiswap MATIC] WMATIC-WBTC,2021-07-06,65.76,64.76,63.33,61.08,57.02,66.78,68.38,71.21,77.66,68.59,67.53,66.01,63.61,59.30,69.68,71.39,74.42,81.34,13046,12760,7993
[Honeyswap MATIC] LINK-DAI,2021-07-06,657.10,64.00,27.19,13.88,7.01,-79.48,-29.64,-14.49,-7.17,4.84,4.53,4.14,3.61,2.87,5.20,5.85,7.37,15.42,21,2916,154
[Sushiswap MATIC] WMATIC-LINK,2021-07-06,63.83,58.41,51.81,43.60,33.10,70.37,83.13,119.13,891.03,72.63,66.23,58.50,48.96,36.93,80.40,95.78,140.58,2179.64,2154,2068,1372
[Sushiswap MATIC] SUSHI-WMATIC,2021-07-06,58.20,58.17,58.12,58.03,57.86,58.24,58.29,58.38,58.55,76.04,76.00,75.92,75.81,75.57,76.09,76.16,76.28,76.53,337281,318020,220853
[Sushiswap MATIC] WMATIC-USDT,2021-07-06,61.78,58.11,53.36,46.96,37.88,65.94,73.36,90.27,167.49,85.77,80.61,73.94,64.98,52.30,91.63,102.11,126.13,238.23,3168,3125,2447
[Quickswap MATIC] SUSHI-WMATIC,2021-07-06,88.68,48.91,29.24,17.51,9.71,474.53,-85.87,-28.93,-12.44,141.01,75.89,44.83,26.65,14.72,993.96,-123.12,-42.85,-18.60,245,233,300
[Sushiswap MATIC] SUSHI-WETH,2021-07-06,44.34,44.33,44.32,44.30,44.26,44.35,44.36,44.38,44.42,69.93,69.92,69.90,69.86,69.80,69.94,69.96,70.00,70.07,1095265,1036305,661827
